＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝  
ex5:  
特徴量を新たに加える
・onehot
・ある遺伝子集合の平均

後処理  
・近傍

とりあえず突っ込んだ
cv score : 0.016759827614837304 (control removed)
cv score : 0.01544

何もなし
cv score : 0.016659014838521283
cv score : 0.01535365993431876

random
cv score : 0.016784831050276998 
random(:400)
cv score : 0.016759801481980134

In [ ]:
import warnings, random, os, sys, tqdm, time
sys.path.append("../")
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from gplearn.genetic import SymbolicTransformer

from sklearn.decomposition import PCA
from sklearn.feature_selection import VarianceThreshold
from sklearn.metrics import log_loss
from sklearn.preprocessing import StandardScaler, MinMaxScaler, QuantileTransformer, RobustScaler
from sklearn.cluster import KMeans
import scipy
from scipy.spatial import distance
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold


import torch
from torch import nn
import torch.optim as optim
import torch.nn.functional as F
from torch.optim.lr_scheduler import ReduceLROnPlateau

from pytorch_tabnet.tab_model import TabNetRegressor
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

pd.set_option("display.max_columns", 1200)
pd.set_option("display.max_rows", 1200)
%matplotlib inline

In [ ]:
def metric(y_true, y_pred):
    res = []
    for i in range(0, y_true.shape[1]):
        y = y_true[:,i]
        pred = y_pred[:,i]
        res.append(log_loss(y, pred))
    return np.mean(res)

def seed_everything(seed_value):
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    os.environ['PYTHONHASHSEED'] = str(seed_value)
    
    if torch.cuda.is_available(): 
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = True
seed_everything(42)
        
    
def make_scaler(flag, seed):
    if flag == "quantile":
        return QuantileTransformer(n_quantiles=100,random_state=seed, output_distribution="normal")
    elif flag == "gauss":
        return GaussRankScaler()
    elif flag == "standard":
        return StandardScaler()
    elif flag == "minmax":
        return MinMaxScaler()
    elif flag == "robust":
        return RobustScaler()
    
seeds = [0, 1, 2, 3, 4, 5, 6]
seeds = [0]
SCALE = "quantile"

"""
def special_pearsonr(X, Y):
    pearsonr_value = scipy.stats.pearsonr(X, Y)
    if pearsonr_value[0] < 0:
        return abs(pearsonr_value[0])
    else:
        return 1 - pearsonr_value[0]


def sub_pearsonr(X, row_index_1, row_index_2):
    pearsonr_distance = special_pearsonr(X[row_index_1], X[row_index_2])
    return (row_index_1, row_index_2, pearsonr_distance)


def pearsonr_distances(X, Y=None):
    if Y==None:
        # Xだけが入力されていて、Xが2d-arrayの場合
        row_combinations = list(combinations(range(0, len(X)), 2))
        pearsonr_set = [sub_pearsonr(X, index_set_tuple[0], index_set_tuple[1]) for index_set_tuple in row_combinations]
        matrix_source_data = pearsonr_set + map(copy_inverse_index, pearsonr_set)

        row = [t[0] for t in matrix_source_data]
        col = [t[1] for t in matrix_source_data]
        data = [t[2] for t in matrix_source_data]

        pearsonr_matrix = special_pearsonr((data, (row, col)), (X.shape[0], X.shape[0]))
        return pearsonr_matrix.toarray()

    elif len(X.shape)==1 and len(Y.shape)==2:
        # Xが１サンプルでYが複数サンプルの場合
        # returnは1 sampleのarray
        pearsonr_set = numpy.array([special_pearsonr(X, Y[y_sample_index]) for y_sample_index in range(0, Y.shape[0])])

        return pearsonr_set

    elif len(X.shape)==2 and len(Y.shape)==2:
        # 1行には、X[i]とY[j] in n(Y)の距離を記録して返す
        # X[i]とYのすべての距離を計算する関数
        pearsonr_x_and_all_y = lambda X, Y: numpy.array([special_pearsonr(X, Y[y_sample_index]) for y_sample_index in range(0, Y.shape[0])])
        pearsonr_divergence_set = numpy.array([pearsonr_x_and_all_y(X[x_i], Y) for x_i in range(0, X.shape[0])])
        return pearsonr_divergence_set
    else:
        raise Exception("Exception case caused")


# 入力を保証するため、ユークリッド距離関数と同じパラメタを持つ関数を用意する
def new_euclidean_distances(X, Y=None, Y_norm_squared=None, squared=False):
    return pearsonr_distances(X, Y)

from sklearn.cluster import k_means_

# ここで上書き！
k_means_.euclidean_distances = new_euclidean_distances

"""

In [ ]:
# g772, c100, 206クラス、402クラスの分類

train_df = pd.read_csv("../../../Data/Raw/train_features.csv")
test_df = pd.read_csv("../../../Data/Raw/test_features.csv")
#pub_test_df = pd.read_csv("../input/moapublictest/test_features.csv")
pub_test_df = pd.read_csv("../../../Data/Raw/test_features.csv")
drug_df = pd.read_csv("../../../Data/Raw/train_drug.csv")#

y = pd.read_csv("../../../Data/Raw/train_targets_scored.csv")
y_non = pd.read_csv("../../../Data/Raw/train_targets_nonscored.csv")
y_all = pd.concat([y, y_non.drop("sig_id", axis=1)], axis=1)
y = y.merge(drug_df, on='sig_id', how='left') #

GENES = [col for col in train_df.columns if col.startswith("g-")]
CELLS = [col for col in train_df.columns if col.startswith("c-")]
BIOS = GENES + CELLS


SCORED_MOAS = [col for col in y.columns if col != "sig_id" and col != "drug_id"]#
NONSCORED_MOAS = [col for col in y_non.columns if col != "sig_id"]
ALL_MOAS = SCORED_MOAS + NONSCORED_MOAS


TR_SIZE = train_df.shape[0]
TE_SIZE = test_df.shape[0]

train_nonvehicle_index = train_df[train_df["cp_type"] != "ctl_vehicle"].index
test_nonvehicle_index = test_df[test_df["cp_type"] != "ctl_vehicle"].index

train_df["time_dose"] = train_df["cp_time"].astype(str) + " * " + train_df["cp_dose"]
test_df["time_dose"] = test_df["cp_time"].astype(str) + " * " + test_df["cp_dose"]
pub_test_df["time_dose"] = pub_test_df["cp_time"].astype(str) + " * " + pub_test_df["cp_dose"]

# remove cp_type = ctl_vehicle
mask = train_df["cp_type"] != "ctl_vehicle"
train_df = train_df[mask].drop("cp_type", axis=1).reset_index(drop=True)
test_df = test_df[test_df["cp_type"] != "ctl_vehicle"].drop("cp_type", axis=1).reset_index(drop=True)
pub_test_df = pub_test_df[pub_test_df["cp_type"] != "ctl_vehicle"].drop("cp_type", axis=1).reset_index(drop=True)
y_nonv = y[mask].reset_index(drop=True)#

scored = y_nonv.copy()#
y_nonv.drop("drug_id", axis=1, inplace=True)#
y.drop("drug_id", axis=1, inplace=True)#

TR_NONV_SIZE = train_df.shape[0]
TE_NONV_SHAPE = test_df.shape[0]

In [ ]:
from pytorch_tabnet.metrics import Metric
class LogitsLogLoss(Metric):
    """
    LogLoss with sigmoid applied
    """

    def __init__(self):
        self._name = "logits_ll"
        self._maximize = False

    def __call__(self, y_true, y_pred):
        """
        Compute LogLoss of predictions.

        Parameters
        ----------
        y_true: np.ndarray
            Target matrix or vector
        y_score: np.ndarray
            Score matrix or vector

        Returns
        -------
            float
            LogLoss of predictions vs targets.
        """
        logits = 1 / (1 + np.exp(-y_pred))
        aux = (1 - y_true) * np.log(1 - logits + 1e-15) + y_true * np.log(logits + 1e-15)
        return np.mean(-aux)
    
class LabelSmoothing(nn.Module):
    """
    NLL loss with label smoothing.
    """
    def __init__(self, smoothing=0.0, n_cls=2):
        """
        Constructor for the LabelSmoothing module.
        :param smoothing: label smoothing factor
        """
        super(LabelSmoothing, self).__init__()
        self.confidence = 1.0 - smoothing + smoothing / n_cls
        self.smoothing = smoothing / n_cls

    def forward(self, x, target):
        probs = torch.nn.functional.sigmoid(x,)
        # ylogy + (1-y)log(1-y)
        #with torch.no_grad():
        target1 = self.confidence * target + (1-target) * self.smoothing
        #print(target1.cpu())
        loss = -(torch.log(probs+1e-15) * target1 + (1-target1) * torch.log(1-probs+1e-15))
        #print(loss.cpu())
        #nll_loss = -logprobs.gather(dim=-1, index=target.unsqueeze(1))
        #nll_loss = nll_loss.squeeze(1)
        #smooth_loss = -logprobs.mean(dim=-1)
        #loss = self.confidence * nll_loss + self.smoothing * smooth_loss
        return loss.mean()


In [ ]:
# prod
#prod_p_cols = [['g-722', 'g-655', 'g-707'], ['g-707', 'g-65', 'g-392'], ['g-169', 'g-484', 'g-338'], ['g-417', 'g-100', 'g-707'], ['g-38', 'g-100', 'g-707'], ['g-310', 'g-744', 'g-707'], ['g-100', 'g-0', 'g-123'], ['g-38', 'g-100', 'g-744'], ['g-328', 'g-707', 'g-158'], ['g-100', 'g-744', 'g-38'], ['g-310', 'g-744', 'g-707'], ['g-491', 'g-100', 'g-38'], ['g-135', 'g-392', 'g-512'], ['g-131', 'g-38', 'g-708'], ['g-180', 'g-624', 'g-613'], ['g-707', 'g-133', 'g-392'], ['g-69', 'g-707', 'g-100'], ['g-392', 'g-731', 'g-707'], ['g-759', 'g-392', 'g-65'], ['g-544', 'g-425', 'g-707'], ['g-69', 'g-608', 'g-417'], ['g-441', 'g-703', 'g-491'], ['g-712', 'g-310', 'g-328'], ['g-624', 'g-615', 'g-189'], ['g-57', 'g-729', 'g-130'], ['g-146', 'g-466', 'g-762'], ['g-308', 'g-495', 'g-712'], ['g-181', 'g-707', 'g-38'], ['g-392', 'g-731', 'g-131'], ['g-349', 'g-750', 'g-91'], ['g-541', 'g-748', 'g-38'], ['g-91', 'g-100', 'g-478'], ['g-635', 'g-514', 'g-302'], ['g-419', 'g-676', 'g-130'], ['g-744', 'g-131', 'g-100'], ['g-707', 'g-158', 'g-100'], ['g-127', 'g-749', 'g-380'], ['g-392', 'g-731', 'g-100'], ['g-144', 'g-123', 'g-86'], ['g-732', 'g-744', 'g-707'], ['g-744', 'g-731', 'g-100'], ['g-731', 'g-158', 'g-38'], ['g-158', 'g-100', 'g-707'], ['g-208', 'g-707', 'g-731'], ['g-38', 'g-392', 'g-707'], ['g-744', 'g-721', 'g-707'], ['g-162', 'g-157', 'g-178'], ['g-326', 'g-707', 'g-449'], ['g-504', 'g-392', 'g-707'], ['g-729', 'g-182', 'g-208'], ['g-744', 'g-608', 'g-100'], ['g-452', 'g-391', 'g-413'], ['g-714', 'g-452', 'g-658'], ['g-100', 'g-392', 'g-707'], ['g-640', 'g-266', 'g-310'], ['g-91', 'g-145', 'g-208'], ['g-744', 'g-158', 'g-392'], ['g-16', 'g-714', 'g-707'], ['g-310', 'g-13', 'g-100'], ['g-478', 'g-468', 'g-310'], ['g-689', 'g-100', 'g-707'], ['g-208', 'g-714', 'g-707'], ['g-38', 'g-158', 'g-707'], ['g-484', 'g-392', 'g-544'], ['g-392', 'g-484', 'g-74'], ['g-95', 'g-170', 'g-707'], ['g-91', 'g-130', 'g-707'], ['g-131', 'g-208', 'g-392'], ['g-417', 'g-248', 'g-744'], ['g-707', 'g-607', 'g-358'], ['g-392', 'g-707', 'g-158'], ['g-31', 'g-328', 'g-460'], ['g-576', 'g-666', 'g-608'], ['g-368', 'g-402', 'g-707'], ['g-512', 'g-594', 'g-38'], ['g-38', 'g-707', 'g-158'], ['g-392', 'g-100', 'g-707'], ['g-91', 'g-100', 'g-707'], ['g-158', 'g-38', 'g-744'], ['g-744', 'g-707', 'g-100'], ['g-654', 'g-143', 'g-377'], ['g-131', 'g-100', 'g-170'], ['g-699', 'g-235', 'g-707'], ['g-744', 'g-392', 'g-38'], ['g-682', 'g-592', 'g-707'], ['g-391', 'g-666', 'g-514'], ['g-143', 'g-231', 'g-265'], ['g-478', 'g-442', 'g-270'], ['g-608', 'g-162', 'g-11'], ['g-134', 'g-54', 'g-762'], ['g-145', 'g-201', 'g-208'], ['g-413', 'g-310', 'g-707'], ['g-744', 'g-38', 'g-100'], ['g-413', 'g-707', 'g-69'], ['g-123', 'g-731', 'g-100'], ['g-712', 'g-208', 'g-38'], ['g-731', 'g-707', 'g-100'], ['g-38', 'g-744', 'g-100'], ['g-576', 'g-452', 'g-392'], ['g-441', 'g-157', 'g-392'], ['g-596', 'g-744', 'g-707'], ['g-38', 'g-392', 'g-744'], ['g-392', 'g-69', 'g-654'], ['g-123', 'g-744', 'g-38'], ['g-417', 'g-46', 'g-181'], ['g-731', 'g-100', 'g-707'], ['g-484', 'g-707', 'g-158'], ['g-744', 'g-707', 'g-100'], ['g-100', 'g-158', 'g-707'], ['g-200', 'g-707', 'g-592'], ['g-215', 'g-392', 'g-330'], ['g-383', 'g-576', 'g-514'], ['g-689', 'g-69', 'g-100'], ['g-645', 'g-123', 'g-514'], ['g-697', 'g-200', 'g-608'], ['g-91', 'g-100', 'g-392'], ['g-38', 'g-707', 'g-158'], ['g-231', 'g-100', 'g-707'], ['g-573', 'g-127', 'g-70'], ['g-178', 'g-592', 'g-391'], ['g-368', 'g-100', 'g-707'], ['g-200', 'g-729', 'g-648'], ['g-723', 'g-731', 'g-100'], ['g-130', 'g-745', 'g-158'], ['g-208', 'g-131', 'g-100'], ['g-392', 'g-38', 'g-91'], ['g-731', 'g-744', 'g-158'], ['g-437', 'g-158', 'g-91'], ['g-707', 'g-335', 'g-116'], ['g-100', 'g-731', 'g-707'], ['g-44', 'g-714', 'g-707'], ['g-392', 'g-413', 'g-707'], ['g-65', 'g-392', 'g-158'], ['g-592', 'g-157', 'g-197'], ['g-740', 'g-762', 'g-170'], ['g-137', 'g-188', 'g-181'], ['g-270', 'g-170', 'g-100'], ['g-260', 'g-190', 'g-130'], ['g-718', 'g-722', 'g-707'], ['g-146', 'g-270', 'g-744'], ['g-505', 'g-391', 'g-38'], ['g-100', 'g-731', 'g-707'], ['g-84', 'g-65', 'g-707'], ['g-65', 'g-484', 'g-707'], ['g-328', 'g-28', 'g-392'], ['g-624', 'g-608', 'g-707'], ['g-635', 'g-158', 'g-707'], ['g-419', 'g-143', 'g-640'], ['g-744', 'g-707', 'g-100'], ['g-158', 'g-707', 'g-12'], ['g-744', 'g-707', 'g-38'], ['g-666', 'g-466', 'g-191'], ['g-38', 'g-328', 'g-417'], ['g-328', 'g-38', 'g-100'], ['g-417', 'g-392', 'g-707'], ['g-130', 'g-608', 'g-707'], ['g-380', 'g-38', 'g-707'], ['g-707', 'g-391', 'g-624'], ['g-613', 'g-330', 'g-106'], ['g-131', 'g-100', 'g-744'], ['g-181', 'g-512', 'g-270'], ['g-181', 'g-100', 'g-392'], ['g-744', 'g-417', 'g-100'], ['g-158', 'g-208', 'g-159'], ['g-88', 'g-643', 'g-436'], ['g-484', 'g-158', 'g-707'], ['g-100', 'g-744', 'g-392'], ['g-188', 'g-689', 'g-290'], ['g-158', 'g-417', 'g-11'], ['g-328', 'g-220', 'g-541'], ['g-243', 'g-259', 'g-744'], ['g-392', 'g-313', 'g-422'], ['g-707', 'g-131', 'g-123'], ['g-208', 'g-100', 'g-707'], ['g-158', 'g-417', 'g-707'], ['g-392', 'g-313', 'g-389'], ['g-16', 'g-707', 'g-100'], ['g-231', 'g-707', 'g-392'], ['g-725', 'g-712', 'g-640'], ['g-38', 'g-158', 'g-707'], ['g-201', 'g-745', 'g-599'], ['g-419', 'g-158', 'g-714'], ['g-173', 'g-596', 'g-737'], ['g-326', 'g-248', 'g-600'], ['g-131', 'g-100', 'g-158'], ['g-654', 'g-160', 'g-707'], ['g-147', 'g-731', 'g-721'], ['g-378', 'g-358', 'g-643'], ['g-100', 'g-38', 'g-744'], ['g-166', 'g-635', 'g-430'], ['g-417', 'g-38', 'g-328'], ['g-208', 'g-100', 'g-707'], ['g-38', 'g-744', 'g-100'], ['g-229', 'g-711', 'g-91'], ['g-248', 'g-38', 'g-338'], ['g-38', 'g-417', 'g-707'], ['g-731', 'g-100', 'g-707'], ['g-106', 'g-744', 'g-91'], ['g-707', 'g-624', 'g-455'], ['g-310', 'g-707', 'g-290'], ['g-131', 'g-170', 'g-158'], ['g-146', 'g-270', 'g-158'], ['g-529', 'g-707', 'g-74']]
#prod_n_cols = [['g-431', 'g-597', 'g-489'], ['g-239', 'g-113', 'g-50'], ['g-370', 'g-431', 'g-656'], ['g-568', 'g-508', 'g-760'], ['g-370', 'g-508', 'g-37'], ['g-228', 'g-72', 'g-67'], ['g-50', 'g-37', 'g-250'], ['g-508', 'g-50', 'g-411'], ['g-128', 'g-370', 'g-429'], ['g-50', 'g-672', 'g-37'], ['g-508', 'g-37', 'g-489'], ['g-50', 'g-705', 'g-298'], ['g-771'], ['g-67', 'g-644', 'g-113'], ['g-50', 'g-37', 'g-36'], ['g-151', 'g-495', 'g-234'], ['g-276', 'g-178', 'g-428'], ['g-644', 'g-370', 'g-411'], ['g-370', 'g-568', 'g-50'], ['g-50', 'g-37', 'g-332'], ['g-375', 'g-644', 'g-271'], ['g-571', 'g-503', 'g-370'], ['g-56', 'g-161', 'g-298'], ['g-508', 'g-37', 'g-370'], ['g-537', 'g-487', 'g-719'], ['g-211', 'g-75', 'g-501'], ['g-503', 'g-477', 'g-489'], ['g-508', 'g-113', 'g-231'], ['g-113', 'g-375', 'g-75'], ['g-50', 'g-332', 'g-37'], ['g-113', 'g-75', 'g-178'], ['g-644', 'g-178', 'g-760'], ['g-50', 'g-72', 'g-37'], ['g-653', 'g-202', 'g-378'], ['g-300', 'g-247', 'g-584'], ['g-37', 'g-50', 'g-98'], ['g-50', 'g-58', 'g-332'], ['g-411', 'g-674', 'g-299'], ['g-672', 'g-50', 'g-508'], ['g-128', 'g-370', 'g-644'], ['g-37', 'g-228', 'g-202'], ['g-508', 'g-760', 'g-406'], ['g-75', 'g-113', 'g-228'], ['g-508', 'g-50', 'g-370'], ['g-72', 'g-370', 'g-50'], ['g-370', 'g-75', 'g-37'], ['g-399', 'g-644', 'g-739'], ['g-674', 'g-555', 'g-713'], ['g-610', 'g-537', 'g-323'], ['g-276', 'g-113', 'g-555'], ['g-421', 'g-612', 'g-464'], ['g-370', 'g-50', 'g-503'], ['g-508', 'g-151', 'g-267'], ['g-258', 'g-644', 'g-370'], ['g-370', 'g-276', 'g-75'], ['g-50', 'g-560', 'g-630'], ['g-332', 'g-178', 'g-508'], ['g-50', 'g-37', 'g-75'], ['g-370', 'g-558', 'g-371'], ['g-37', 'g-62', 'g-271'], ['g-75', 'g-50', 'g-375'], ['g-37', 'g-421', 'g-72'], ['g-370', 'g-684', 'g-508'], ['g-50', 'g-508', 'g-113'], ['g-50', 'g-298', 'g-37'], ['g-239', 'g-370', 'g-739'], ['g-705', 'g-50', 'g-298'], ['g-37', 'g-5', 'g-332'], ['g-508', 'g-37', 'g-72'], ['g-513', 'g-508', 'g-128'], ['g-555', 'g-281', 'g-172'], ['g-50', 'g-37', 'g-489'], ['g-370', 'g-228', 'g-644'], ['g-370', 'g-719', 'g-508'], ['g-298', 'g-477', 'g-644'], ['g-257', 'g-50', 'g-72'], ['g-75', 'g-370', 'g-672'], ['g-508', 'g-113', 'g-370'], ['g-332', 'g-58', 'g-37'], ['g-508', 'g-37', 'g-672'], ['g-50', 'g-37', 'g-672'], ['g-681', 'g-272', 'g-131'], ['g-771'], ['g-50', 'g-37', 'g-672'], ['g-370', 'g-571', 'g-438'], ['g-508', 'g-332', 'g-50'], ['g-370', 'g-400', 'g-300'], ['g-300', 'g-284', 'g-495'], ['g-234', 'g-761', 'g-555'], ['g-257', 'g-672', 'g-477'], ['g-370', 'g-227', 'g-653'], ['g-238', 'g-399', 'g-759'], ['g-508', 'g-228', 'g-656'], ['g-370', 'g-618', 'g-644'], ['g-508', 'g-228', 'g-113'], ['g-67', 'g-571', 'g-653'], ['g-37', 'g-228', 'g-489'], ['g-67', 'g-178', 'g-113'], ['g-202', 'g-370', 'g-421'], ['g-50', 'g-113', 'g-672'], ['g-370', 'g-399', 'g-98'], ['g-401', 'g-58', 'g-202'], ['g-508', 'g-370', 'g-58'], ['g-75', 'g-37', 'g-50'], ['g-300', 'g-370', 'g-568'], ['g-508', 'g-37', 'g-50'], ['g-234', 'g-271', 'g-632'], ['g-67', 'g-760', 'g-50'], ['g-113', 'g-75', 'g-489'], ['g-37', 'g-50', 'g-508'], ['g-508', 'g-67', 'g-37'], ['g-555', 'g-492', 'g-653'], ['g-113', 'g-250', 'g-325'], ['g-705', 'g-178', 'g-284'], ['g-332', 'g-50', 'g-571'], ['g-508', 'g-67', 'g-276'], ['g-375', 'g-644', 'g-113'], ['g-298', 'g-257', 'g-332'], ['g-75', 'g-50', 'g-653'], ['g-204', 'g-558', 'g-202'], ['g-587', 'g-644', 'g-17'], ['g-204', 'g-567', 'g-526'], ['g-571', 'g-568', 'g-760'], ['g-653', 'g-555', 'g-678'], ['g-202', 'g-50', 'g-760'], ['g-50', 'g-298', 'g-332'], ['g-50', 'g-37', 'g-59'], ['g-67', 'g-113', 'g-508'], ['g-75', 'g-298', 'g-508'], ['g-508', 'g-50', 'g-298'], ['g-487', 'g-285', 'g-232'], ['g-370', 'g-644', 'g-50'], ['g-370', 'g-575', 'g-477'], ['g-50', 'g-257', 'g-37'], ['g-508', 'g-644', 'g-113'], ['g-513', 'g-591', 'g-300'], ['g-375', 'g-681', 'g-492'], ['g-571', 'g-234', 'g-412'], ['g-37', 'g-58', 'g-228'], ['g-17', 'g-115', 'g-300'], ['g-323', 'g-506', 'g-168'], ['g-771'], ['g-743', 'g-497', 'g-595'], ['g-375', 'g-370', 'g-508'], ['g-50', 'g-37', 'g-568'], ['g-50', 'g-438', 'g-439'], ['g-370', 'g-67', 'g-58'], ['g-477', 'g-568', 'g-761'], ['g-370', 'g-267', 'g-705'], ['g-234', 'g-58', 'g-520'], ['g-234', 'g-508', 'g-595'], ['g-37', 'g-50', 'g-202'], ['g-37', 'g-644', 'g-489'], ['g-477', 'g-72', 'g-37'], ['g-506', 'g-719', 'g-300'], ['g-370', 'g-508', 'g-37'], ['g-508', 'g-50', 'g-37'], ['g-370', 'g-50', 'g-113'], ['g-370', 'g-300', 'g-487'], ['g-568', 'g-276', 'g-719'], ['g-421', 'g-434', 'g-644'], ['g-595', 'g-176', 'g-487'], ['g-50', 'g-672', 'g-489'], ['g-508', 'g-50', 'g-250'], ['g-250', 'g-489', 'g-439'], ['g-72', 'g-644', 'g-406'], ['g-185', 'g-37', 'g-672'], ['g-300', 'g-227', 'g-591'], ['g-332', 'g-37', 'g-370'], ['g-50', 'g-37', 'g-489'], ['g-367', 'g-438', 'g-144'], ['g-37', 'g-50', 'g-257'], ['g-276', 'g-653', 'g-291'], ['g-50', 'g-761', 'g-672'], ['g-477', 'g-571', 'g-585'], ['g-370', 'g-113', 'g-508'], ['g-508', 'g-370', 'g-332'], ['g-508', 'g-75', 'g-113'], ['g-58', 'g-186', 'g-477'], ['g-202', 'g-479', 'g-660'], ['g-477', 'g-332', 'g-36'], ['g-239', 'g-42', 'g-234'], ['g-508', 'g-370', 'g-37'], ['g-37', 'g-300', 'g-370'], ['g-370', 'g-300', 'g-96'], ['g-5', 'g-429', 'g-299'], ['g-477', 'g-497', 'g-487'], ['g-152', 'g-50', 'g-75'], ['g-375', 'g-477', 'g-585'], ['g-228', 'g-411', 'g-67'], ['g-190', 'g-475', 'g-628'], ['g-202', 'g-37', 'g-477'], ['g-595', 'g-276', 'g-75'], ['g-72', 'g-75', 'g-508'], ['g-370', 'g-508', 'g-50'], ['g-37', 'g-75', 'g-121'], ['g-503', 'g-761', 'g-50'], ['g-595', 'g-486', 'g-72'], ['g-508', 'g-67', 'g-370'], ['g-50', 'g-508', 'g-75'], ['g-508', 'g-553', 'g-72'], ['g-595', 'g-665', 'g-131'], ['g-705', 'g-375', 'g-704'], ['g-75', 'g-228', 'g-508'], ['g-508', 'g-37', 'g-644'], ['g-477', 'g-50', 'g-72']]
# 上位500こ
prod_cols = [['g-145', 'g-201', 'g-208'], ['g-370', 'g-508', 'g-37'], ['g-38', 'g-392', 'g-707'], ['g-328', 'g-28', 'g-392'], ['g-441', 'g-157', 'g-392'], ['g-181', 'g-100', 'g-392'], ['g-67', 'g-760', 'g-50'], ['g-731', 'g-100', 'g-707'], ['g-478', 'g-468', 'g-310'], ['g-91', 'g-145', 'g-208'], ['g-106', 'g-744', 'g-91'], ['g-131', 'g-208', 'g-392'], ['g-144', 'g-123', 'g-86'], ['g-228', 'g-72', 'g-67'], ['g-31', 'g-328', 'g-460'], ['g-392', 'g-731', 'g-100'], ['g-732', 'g-744', 'g-707'], ['g-705', 'g-375', 'g-704'], ['g-508', 'g-50', 'g-411'], ['g-234', 'g-58', 'g-520'], ['g-503', 'g-761', 'g-50'], ['g-113', 'g-75', 'g-178'], ['g-50', 'g-508', 'g-113'], ['g-113', 'g-375', 'g-75'], ['g-576', 'g-452', 'g-392'], ['g-50', 'g-37', 'g-36'], ['g-707', 'g-133', 'g-392'], ['g-484', 'g-392', 'g-544'], ['g-508', 'g-67', 'g-370'], ['g-123', 'g-731', 'g-100'], ['g-298', 'g-477', 'g-644'], ['g-72', 'g-370', 'g-50'], ['g-67', 'g-178', 'g-113'], ['g-744', 'g-608', 'g-100'], ['g-91', 'g-100', 'g-707'], ['g-37', 'g-228', 'g-202'], ['g-37', 'g-300', 'g-370'], ['g-234', 'g-508', 'g-595'], ['g-596', 'g-744', 'g-707'], ['g-300', 'g-227', 'g-591'], ['g-135', 'g-392', 'g-512'], ['g-731', 'g-744', 'g-158'], ['g-69', 'g-707', 'g-100'], ['g-276', 'g-653', 'g-291'], ['g-624', 'g-615', 'g-189'], ['g-181', 'g-707', 'g-38'], ['g-72', 'g-75', 'g-508'], ['g-231', 'g-707', 'g-392'], ['g-508', 'g-37', 'g-72'], ['g-725', 'g-712', 'g-640'], ['g-67', 'g-644', 'g-113'], ['g-508', 'g-228', 'g-656'], ['g-185', 'g-37', 'g-672'], ['g-370', 'g-50', 'g-503'], ['g-201', 'g-745', 'g-599'], ['g-332', 'g-50', 'g-571'], ['g-50', 'g-37', 'g-59'], ['g-508', 'g-113', 'g-231'], ['g-707', 'g-158', 'g-100'], ['g-257', 'g-50', 'g-72']]

#prod_cols = [['c-36', 'c-76'], ['c-12', 'c-91'], ['g-170', 'c-10'], ['g-701', 'c-64'], ['g-70', 'c-9'], ['g-569', 'c-44'], ['g-315', 'g-392'], ['c-58', 'c-65'], ['g-20', 'g-63'], ['g-682', 'c-90'], ['g-59', 'g-674'], ['g-667', 'c-98'], ['g-336', 'g-418'], ['g-41', 'g-701'], ['g-327', 'c-3'], ['g-243', 'g-370'], ['g-677', 'c-23'], ['g-373', 'g-553'], ['g-314', 'g-473'], ['g-170', 'c-45'], ['g-251', 'g-354'], ['g-148', 'g-377'], ['g-533', 'c-95'], ['g-719', 'c-31'], ['g-186', 'c-85']]
for cols in prod_cols:
    name = "prod-" + " * ".join(cols)
    train_df[name] = train_df[cols].mean(axis=1)
    test_df[name] = test_df[cols].mean(axis=1)
    pub_test_df[name] = pub_test_df[cols].mean(axis=1)

PRODS = [col for col in train_df.columns if col.startswith("prod-")]

In [ ]:
"""
for i in range(400):
    cols = list(np.array(BIOS)[sorted(np.random.randint(0, len(BIOS), 2))])
    name = "prod-" + " * ".join(cols)
    train_df[name] = train_df[cols].mean(axis=1)
    test_df[name] = test_df[cols].mean(axis=1)
    pub_test_df[name] = pub_test_df[cols].mean(axis=1)
PRODS = [col for col in train_df.columns if col.startswith("prod-")]     
"""

In [ ]:
#out fold preprocessing

#variance threshold

VAR_THRESHOLD = 0.8
drop_cols = []
temp = pd.concat([train_df, pub_test_df])
#for col in BIOS:
for col in BIOS+PRODS:
    if temp[col].var() <= VAR_THRESHOLD:
        drop_cols.append(col)
print("drop cols num : {}".format(len(drop_cols)))
train_df.drop(columns=drop_cols, inplace=True)
test_df.drop(columns=drop_cols, inplace=True)
pub_test_df.drop(columns=drop_cols, inplace=True)

GENES_ = [col for col in train_df.columns if col.startswith("g-")]
CELLS_ = [col for col in train_df.columns if col.startswith("c-")]
BIOS_ = GENES_ + CELLS_
PRODS = [col for col in train_df.columns if col.startswith("prod-")]
        
del temp

# onehot encode of categorical feature and drop
#drop_cols = ["cp_time", "cp_dose", "time_dose"]
#train_df = pd.concat([pd.get_dummies(train_df["time_dose"], prefix="onehot", drop_first=True), train_df.drop(drop_cols, axis=1) ], axis=1)
#test_df = pd.concat([pd.get_dummies(test_df["time_dose"], prefix="onehot", drop_first=True), test_df.drop(drop_cols, axis=1) ], axis=1)
#pub_test_df = pd.concat([pd.get_dummies(pub_test_df["time_dose"], prefix="onehot", drop_first=True), pub_test_df.drop(drop_cols, axis=1) ], axis=1)


# aggregation feature
print("agg")
for df in [train_df, pub_test_df, test_df]:
    df["sum-g"] = df[GENES_].sum(axis=1)
    df["mean-g"] = df[GENES_].mean(axis=1)
    df["std-g"] = df[GENES_].std(axis=1)
    df["kurt-g"] = df[GENES_].kurt(axis=1)
    df["skew-g"] = df[GENES_].skew(axis=1)
    df["sum-c"] = df[CELLS_].sum(axis=1)
    df["mean-c"] = df[CELLS_].mean(axis=1)
    df["std-c"] = df[CELLS_].std(axis=1)
    df["kurt-c"] = df[CELLS_].kurt(axis=1)
    df["skew-c"] = df[CELLS_].skew(axis=1)
    df["sum-gc"] = df[BIOS_].sum(axis=1)
    df["mean-gc"] = df[BIOS_].mean(axis=1)
    df["std-gc"] = df[BIOS_].std(axis=1)
    df["kurt-gc"] = df[BIOS_].kurt(axis=1)
    df["skew-gc"] = df[BIOS_].skew(axis=1)


In [ ]:
X = train_df.drop("sig_id", axis=1)
y_nonv = y_nonv.drop("sig_id", axis=1).values
y = y.drop("sig_id", axis=1).values

In [ ]:
MAX_EPOCH = 200
tabnet_params = dict(
    n_d = 21,
    n_a = 21,
    n_steps = 1,
    gamma = 1.3,
    lambda_sparse = 0,
    #lambda_sparse = 1e-4,
    optimizer_fn = optim.Adam,
    optimizer_params = dict(lr = 2e-2, weight_decay = 1e-5),
    mask_type = "entmax",
    scheduler_fn = ReduceLROnPlateau,
    scheduler_params = dict(mode = "min", patience = 3, min_lr = 1e-5, factor = 0.9),

    seed = 42,
    verbose = 10
)


train_preds = np.zeros((X.shape[0], y_nonv.shape[1]))
preds = np.zeros((test_df.shape[0], y_nonv.shape[1]))
imps = []
imp_cols = []
folds = []
test_cv_preds = []

for seed in seeds:
    seed_everything(seed)
    K = 5
    kf = MultilabelStratifiedKFold(n_splits=K, random_state=seed, shuffle=True)
    train_pred = np.zeros(train_preds.shape)
    
    ###############################################################################################
    # LOAD LIBRARIES
    targets = SCORED_MOAS.copy()

    # LOCATE DRUGS
    vc = scored["drug_id"].value_counts()
    vc1 = vc.loc[vc<=18].index.sort_values()
    vc2 = vc.loc[vc>18].index.sort_values()

    # STRATIFY DRUGS 18X OR LESS
    dct1 = {}; dct2 = {}
    skf = MultilabelStratifiedKFold(n_splits=K, shuffle=True, random_state=seed)
    tmp = scored.groupby('drug_id')[targets].mean().loc[vc1]
    for fold,(idxT,idxV) in enumerate( skf.split(tmp,tmp[targets])):
        dd = {k:fold for k in tmp.index[idxV].values} # drug id がどのフォールドに属すか格納
        dct1.update(dd)

    # STRATIFY DRUGS MORE THAN 18X
    skf = MultilabelStratifiedKFold(n_splits=K, shuffle=True, random_state=seed)
    tmp = scored.loc[scored["drug_id"].isin(vc2)].reset_index(drop=True)
    for fold,(idxT,idxV) in enumerate( skf.split(tmp,tmp[targets])):
        dd = {k:fold for k in tmp["sig_id"][idxV].values}
        dct2.update(dd)

    # ASSIGN K
    scored['fold'] = scored.drug_id.map(dct1)
    scored.loc[scored["fold"].isna(),'fold'] = scored.loc[scored["fold"].isna(),'sig_id'].map(dct2)
    scored["fold"] = scored["fold"].astype('int8')
    ###############################################################################################

    

    #for fold, (train_index, valid_index) in enumerate(kf.split(X, y_nonv)):
    for fold in range(K):
        train_index = scored[scored["fold"] != fold].index.to_list()
        valid_index = scored[scored["fold"] == fold].index.to_list()
        print("======================== fold {} ========================".format(fold+1))
        tabnet_params["seed"] = seed+(fold+1)
        folds.append(valid_index)
                
        # split data
        train_X = X.iloc[train_index]
        train_y = y_nonv[train_index]
        valid_X = X.iloc[valid_index]
        valid_y = y_nonv[valid_index]
        test_X = (test_df.drop("sig_id", axis=1))
        pub_test_X = (pub_test_df.drop("sig_id", axis=1))
        
        

        


        # scaler
        print(SCALE)
        scale_cols = BIOS_+PRODS
        scaler = make_scaler(SCALE, seed).fit(train_X.append(pub_test_X)[scale_cols])
        for df in [train_X, valid_X, test_X, pub_test_X]:
            df[scale_cols] = scaler.transform(df[scale_cols])
        
        
        #kmeans
        #print("kmeans gene clustering")
        #train_X["kmeans-g-dist"] = 0
        #valid_X["kmeans-g-dist"] = 0
        #test_X["kmeans-g-dist"] = 0
        #pub_test_X["kmeans-g-dist"] = 0
        #train_X["kmeans-g"] = 0
        #valid_X["kmeans-g"] = 0
        #test_X["kmeans-g"] = 0
        #pub_test_X["kmeans-g"] = 0
        #n_clusters = 10 # peason
        #for val in train_X["time_dose"].unique():
        #    kmeans = KMeans(n_clusters=n_clusters, random_state=seed, init='random').fit(train_X[train_X["time_dose"]==val].append(pub_test_X[pub_test_X["time_dose"]==val])[GENES_]) 
        #    for df in [train_X, valid_X, test_X, pub_test_X]:
        #        df["kmeans-g"][df["time_dose"]==val] = kmeans.predict(df[df["time_dose"]==val][GENES_])
        #        #df["kmeans-g-dist"][df["time_dose"]==val] = df[df["time_dose"]==val].apply(lambda x : 1 / special_pearsonr(kmeans.cluster_centers_[int(x["kmeans-g"])], x[GENES_]), axis=1)
        #        df["kmeans-g-dist"][df["time_dose"]==val] = df[df["time_dose"]==val].apply(lambda x : distance.euclidean(kmeans.cluster_centers_[int(x["kmeans-g"])], x[GENES_]), axis=1).map(np.log10)
                
                
        
        # kmeans
        #print("kmeans gene clustering")
        #n_clusters = 7 # peason
        #n_clusters = 5 # euclid
        #kmeans = KMeans(n_clusters=n_clusters, random_state=seed, init='random').fit(train_X.append(pub_test_X)[GENES_])  
        #for df in [train_X, valid_X, test_X, pub_test_X]:
            #df["kmeans-g"] = kmeans.predict(df[GENES_])
            #df["kmeans-g-dist"] = df.apply(lambda x : 1 / special_pearsonr(kmeans.cluster_centers_[int(x["kmeans-g"])], x[GENES_]), axis=1)
            #df["kmeans-g-dist"] = df.apply(lambda x : distance.euclidean(kmeans.cluster_centers_[int(x["kmeans-g"])], x[GENES_]), axis=1)
            #df["kmeans-g-dist"] = df["kmeans-g-dist"].map(np.log10)

        #train_X = pd.concat([pd.get_dummies(train_X["kmeans-g"], prefix="kmeans-g", drop_first=True), train_X.drop("kmeans-g", axis=1) ], axis=1)
        #valid_X = pd.concat([pd.get_dummies(valid_X["kmeans-g"], prefix="kmeans-g", drop_first=True), valid_X.drop("kmeans-g", axis=1) ], axis=1)
        #test_X = pd.concat([pd.get_dummies(test_X["kmeans-g"], prefix="kmeans-g", drop_first=True), test_X.drop("kmeans-g", axis=1) ], axis=1)
        #pub_test_X = pd.concat([pd.get_dummies(pub_test_X["kmeans-g"], prefix="kmeans-g", drop_first=True), pub_test_X.drop("kmeans-g", axis=1) ], axis=1)
        
        
        # kmeans
        #print("kmeans cell clustering")
        #n_clusters = 5
        #kmeans = KMeans(n_clusters=n_clusters, random_state=seed, init='random').fit(train_X.append(pub_test_X)[CELLS_])  
        #for df in [train_X, valid_X, test_X, pub_test_X]:
        #    df["kmeans-c"] = kmeans.predict(df[CELLS_])

        #train_X = pd.concat([pd.get_dummies(train_X["kmeans-c"], prefix="kmeans-c", drop_first=True), train_X.drop("kmeans-c", axis=1) ], axis=1)
        #valid_X = pd.concat([pd.get_dummies(valid_X["kmeans-c"], prefix="kmeans-c", drop_first=True), valid_X.drop("kmeans-c", axis=1) ], axis=1)
        #test_X = pd.concat([pd.get_dummies(test_X["kmeans-c"], prefix="kmeans-c", drop_first=True), test_X.drop("kmeans-c", axis=1) ], axis=1)
        #pub_test_X = pd.concat([pd.get_dummies(pub_test_X["kmeans-c"], prefix="kmeans-c", drop_first=True), pub_test_X.drop("kmeans-c", axis=1) ], axis=1)

        
        
        print("PCA")
        #PCA
        n_decom_g = 80
        n_decom_c = 10
        decom_g_cols = [f"pca_g-{i}" for i in range(n_decom_g)]
        decom_c_cols = [f"pca_c-{i}" for i in range(n_decom_c)]
        
        pca_g = PCA(n_components = n_decom_g, random_state = seed).fit(train_X.append(pub_test_X)[GENES_])
        pca_c = PCA(n_components = n_decom_c, random_state = seed).fit(train_X.append(pub_test_X)[CELLS_])
        
        for df in [train_X, valid_X, test_X, pub_test_X]:
            df[decom_g_cols] = pca_g.transform(df[GENES_])
            df[decom_c_cols] = pca_c.transform(df[CELLS_])
            
        
        
        
        
        
        
        # onehot encode of categorical feature and drop
        #drop_cols = ["cp_time", "cp_dose", "time_dose", "kmeans-g"]
        drop_cols = ["cp_time", "cp_dose", "time_dose"]
        train_X = pd.concat([pd.get_dummies(train_X["time_dose"], prefix="onehot", drop_first=True), train_X.drop(drop_cols, axis=1) ], axis=1)
        valid_X = pd.concat([pd.get_dummies(valid_X["time_dose"], prefix="onehot", drop_first=True), valid_X.drop(drop_cols, axis=1) ], axis=1)
        test_X = pd.concat([pd.get_dummies(test_X["time_dose"], prefix="onehot", drop_first=True), test_X.drop(drop_cols, axis=1) ], axis=1)
        pub_test_X = pd.concat([pd.get_dummies(pub_test_X["time_dose"], prefix="onehot", drop_first=True), pub_test_X.drop(drop_cols, axis=1) ], axis=1)
        print(train_X.shape[1])

        imp_cols = train_X.columns.to_list()
        # prepare data for training
        train_X = train_X.values
        valid_X = valid_X.values
        test_X = test_X.values
        
        
        # model training
        #tabnet_params["scheduler_params"]["steps_per_epoch"] = int(np.ceil(train_X.shape[0]/1024))
        model = TabNetRegressor(**tabnet_params)
        device = torch.device("cuda:1")
        model.device=device

        model.fit(
            X_train=train_X,
            y_train=train_y,
            eval_set=[(valid_X, valid_y)],
            eval_name = ["val"],
            eval_metric = ["logits_ll"],
            max_epochs=MAX_EPOCH,
            patience=20, 
            batch_size=1024, 
            virtual_batch_size=32,
            num_workers=1, 
            drop_last=False,
            loss_fn=LabelSmoothing(1e-5)
        )
        #model.load_model("../input/tabnetweights/tabnet_weights/{}_{}".format(seed, fold))

        val_preds = 1 / (1 + np.exp(-model.predict(valid_X)))
        train_pred[valid_index] +=  val_preds
        
        test_preds = 1 / (1 + np.exp(-model.predict(test_X)))
        preds += test_preds / (K*len(seeds))
        
        name = "{}_{}".format(seed, fold)
        model.save_model("tabnet_weights/"+name)
        imps.append(model.feature_importances_)

        
    print("seed {} , cv score : {}".format(seed, metric(y_nonv, train_pred)))
    train_preds += train_pred/len(seeds)

print("cv score : {}".format(metric(y_nonv, train_preds)))

In [ ]:
imp_cols[850:]

In [ ]:
train_preds2 = np.zeros((TR_SIZE,  y.shape[1]))
train_preds2[train_nonvehicle_index] = train_preds


preds2 = np.zeros((TE_SIZE, y.shape[1]))
preds2[test_nonvehicle_index] = preds

print("cv score : {}".format(metric(y, train_preds2)))

In [ ]:
sub_df = pd.read_csv("../../../Data/Raw/sample_submission.csv")
#sub_df = pd.read_csv("../input/lish-moa/sample_submission.csv")
cols = [col for col in sub_df.columns if col != "sig_id"]
sub_df[cols] = preds2
#sub_df.to_csv("submission.csv", index=False)

In [ ]:
importance = pd.DataFrame(np.array(imps).sum(axis=0), index=imp_cols, columns=['importance']).sort_values('importance', ascending=False)
importance[:500]*100

In [ ]:
importance[500:]

In [ ]:
product_cols = []
size = 400
for col in importance[:size].index:
        if col.startswith("prod-"):
            col = col.replace("prod-", "")
            product_cols.append(col.split(" * "))

In [ ]:
product_cols

In [ ]:
!zip tabnet_weights.zip tabnet_weights/*

In [ ]:
from scipy.spatial import distance
temp_X = pd.read_csv("../../../Data/Raw/train_features.csv")

D1_24 = temp_X[(temp_X["cp_time"] == 24) & (temp_X["cp_dose"] == "D1")].index
D1_48 = temp_X[(temp_X["cp_time"] == 48) & (temp_X["cp_dose"] == "D1")].index
D1_72 = temp_X[(temp_X["cp_time"] == 72) & (temp_X["cp_dose"] == "D1")].index
D2_24 = temp_X[(temp_X["cp_time"] == 24) & (temp_X["cp_dose"] == "D2")].index
D2_48 = temp_X[(temp_X["cp_time"] == 48) & (temp_X["cp_dose"] == "D2")].index
D2_72 = temp_X[(temp_X["cp_time"] == 72) & (temp_X["cp_dose"] == "D2")].index

D1_24_li = []
D1_48_li = []
D1_72_li = []
D2_24_li = []
D2_48_li = []
D2_72_li = []

temp_X = temp_X[GENES].values

ind_d_di = {198 : 27.64044267934251 ,205 : 33.28095914689245 ,219 : 26.293567742286772 ,349 : 32.144217064871754 ,355 : 32.75671799629608 ,359 : 32.405461099926775 ,437 : 25.11853254602351 ,662 : 27.283574492173045 ,758 : 26.439806102764273 ,1036 : 36.948408111645065 ,1136 : 25.47683601518509 ,1373 : 33.9392687045301 ,2034 : 40.5196379718699 ,2130 : 34.08458207580739 ,2564 : 33.43146566974134 ,2647 : 33.68887043662821 ,2836 : 36.04854796893644 ,2939 : 35.11596938312873 ,2956 : 31.56243126231727 ,3139 : 43.71295533889707 ,3203 : 31.18904200394691 ,3223 : 33.70118239355776 ,3276 : 32.61321404813961 ,3356 : 28.320339589604817 ,3452 : 32.85441721909809 ,3618 : 33.18573306434971 ,3643 : 36.54670927105042 ,3673 : 32.246412469025636 ,3715 : 24.950758819234238 ,3772 : 42.121382847554 ,3832 : 31.581639777253443 ,3859 : 34.56812076987165 ,3868 : 33.88130005595618 ,4057 : 37.773757523946635 ,4089 : 32.8298368836978 ,4115 : 34.142369571472 ,4138 : 27.681208302147944 ,4190 : 23.881378338377875 ,4201 : 32.88690807339985 ,4213 : 31.000076698173515 ,4255 : 34.50474421931518 ,4269 : 33.84603755764389 ,4356 : 31.311032534790247 ,4484 : 34.09409546750536 ,4683 : 43.09695176876667 ,4742 : 28.56445864028609 ,4835 : 46.57169334880271 ,4847 : 29.32450603280859 ,5042 : 30.80107192177948 ,5120 : 25.98683374641437 ,5219 : 24.30979050613337 ,5229 : 31.612715539392607 ,5459 : 28.05809706958542 ,5471 : 25.002507506497444 ,5539 : 33.03922631260881 ,5551 : 36.50115438044778 ,5625 : 35.530587615506136 ,5832 : 44.62785678785104 ,5925 : 32.22181957349932 ,5945 : 26.14298505191061 ,5973 : 26.075056831700454 ,6023 : 31.494413803437716 ,6172 : 31.284419290876155 ,6193 : 31.908816011789106 ,6238 : 28.082798870640858 ,6360 : 25.138076093358308 ,6402 : 29.611604598706702 ,6431 : 25.79408382172474 ,6581 : 31.170635370492285 ,6591 : 24.409733116367946 ,6698 : 36.23337744230803 ,6825 : 26.300249184503187 ,7052 : 42.64532049576953 ,7118 : 28.29404738911886 ,7246 : 37.0020277455238 ,7270 : 38.53920083870169 ,7290 : 30.9225312882934 ,7301 : 37.4466173271041 ,7320 : 33.434171179190336 ,7396 : 43.50779255683786 ,7447 : 26.85397863749539 ,7461 : 29.07080733087725 ,7555 : 26.510819849416954 ,7625 : 37.89021071972964 ,7639 : 29.74149163223526 ,7653 : 42.37200231481207 ,7663 : 35.646054034338114 ,7669 : 38.371559686743666 ,7687 : 23.627196546576716 ,7725 : 24.32684436819316 ,7736 : 35.69054864645723 ,7839 : 31.033654444811575 ,7898 : 34.89207684895206 ,8203 : 25.741594660854837 ,8213 : 35.359943589768534 ,8232 : 26.86178971769117 ,8256 : 39.44831673077363 ,8331 : 33.466976602659905 ,8359 : 33.73528992402477 ,8381 : 28.38718401733381 ,8581 : 26.068393195059997 ,8756 : 29.742912319008283 ,8883 : 33.35110791657884 ,8885 : 29.885837876973536 ,9052 : 32.500778588959705 ,9054 : 42.31259549792671 ,9076 : 31.40655287994994 ,9187 : 32.12180681607668 ,9407 : 41.465321504807754 ,9562 : 35.41678848184798 ,9613 : 32.58702261985014 ,9643 : 36.61048256607133 ,9654 : 31.895905516345998 ,9732 : 31.877486529734107 ,9736 : 29.628500084163015 ,9787 : 26.64570320204543 ,9844 : 23.451058893048277 ,9927 : 32.031780082437244 ,9942 : 32.315721578001224 ,9959 : 28.818107378305434 ,10013 : 26.4130456161395 ,10027 : 34.05106155331058 ,10037 : 27.22795414397195 ,10143 : 38.186064953453865 ,10144 : 34.892835002759576 ,10345 : 32.39338902398916 ,10371 : 44.61609468583867 ,10377 : 25.02287770583602 ,10418 : 32.61559371305544 ,10448 : 28.087170023924084 ,10640 : 43.595545871726486 ,10704 : 34.66931912699262 ,10768 : 27.648294658188554 ,10795 : 39.09657681125063 ,10878 : 27.375563743769824 ,10958 : 26.605774339546794 ,11172 : 35.23921742135122 ,11196 : 35.03443399187795 ,11274 : 40.10512986951533 ,11351 : 26.550293610645262 ,11457 : 37.79209269543878 ,11519 : 36.71052626115585 ,11574 : 37.70804071417054 ,11851 : 29.29322905330435 ,11899 : 27.227640413554717 ,11919 : 34.84376868698467 ,12025 : 30.43923276381386 ,12147 : 38.2849904226687 ,12248 : 34.27587022685589 ,12284 : 34.70652246895255 ,12336 : 33.632689885458205 ,12428 : 35.302949114122306 ,12858 : 31.813963630142673 ,12877 : 24.45524969751821 ,13065 : 29.219504381085354 ,13072 : 32.05709698590726 ,13108 : 35.66997380089964 ,13127 : 35.46584888592367 ,13261 : 27.524259487822196 ,13283 : 35.264817663015656 ,13459 : 31.104337911676076 ,13506 : 38.4442468732375 ,13584 : 30.77617919456082 ,13596 : 34.45750721712498 ,13627 : 45.254404919292476 ,13631 : 27.810910913654556 ,13715 : 33.75729134359191 ,13801 : 37.742147143661875 ,13819 : 38.42103427300276 ,13889 : 34.15874676545821 ,13936 : 28.158394902388785 ,13976 : 28.18815995709257 ,14263 : 33.59255567072674 ,14321 : 31.45810289471252 ,14371 : 39.57023369112188 ,14417 : 33.36848928982196 ,14466 : 26.136640892475672 ,14482 : 25.574275359692443 ,14596 : 41.57005202637612 ,14608 : 29.29623175764474 ,14623 : 34.582429465838814 ,14706 : 38.089235999981355 ,14725 : 30.68814021796742 ,14731 : 25.686191452294704 ,14807 : 29.08931288578605 ,14922 : 31.960974837317394 ,14978 : 29.4267409542746 ,15012 : 33.519012702006805 ,15028 : 36.94881744575735 ,15061 : 32.03984963461554 ,15124 : 30.964762097504686 ,15583 : 25.513275820220716 ,15693 : 31.33527784478061 ,15731 : 32.68162752895439 ,15859 : 36.34500948289357 ,15937 : 28.216054022642286 ,16006 : 34.971621575686406 ,16036 : 35.06164013442746 ,16258 : 45.42475492123022 ,16268 : 27.439700196139242 ,16601 : 44.59056949286019 ,16613 : 38.36632394016683 ,16657 : 37.17131677979276 ,16928 : 37.00608770379481 ,17029 : 34.59415728006598 ,17072 : 38.21748285301389 ,17211 : 44.48604500008426 ,17261 : 44.384435680584154 ,17268 : 35.10512195321232 ,17294 : 37.14380466855742 ,17403 : 26.320202818399643 ,17461 : 28.29213417263962 ,17556 : 37.29155363542687 ,17596 : 25.675555104974396 ,17720 : 29.425957803603904 ,17807 : 30.69965608357629 ,17908 : 36.610626923453836 ,18107 : 29.72992736924811 ,18170 : 29.346418441291384 ,18220 : 31.765572217199946 ,18293 : 34.94450856535027 ,18450 : 33.058569638884684 ,18473 : 43.33165196333056 ,18493 : 30.344735554758383 ,18704 : 39.67070021487053 ,18815 : 25.49752813013908 ,18862 : 26.26163627774139 ,18924 : 44.949203946477674 ,19015 : 31.474567135688318 ,19157 : 27.891253603952997 ,19302 : 37.43882607120238 ,19365 : 37.40357115032789 ,19655 : 34.30282498162036 ,19695 : 41.09873298075693 ,19814 : 36.81446311486772 ,20034 : 36.88374017923504 ,20036 : 35.790407700111246 ,20045 : 39.420788790114194 ,20313 : 36.163335008681955 ,20539 : 34.48401240482135 ,20557 : 32.63326830507694 ,20695 : 32.10675785056961 ,20724 : 30.105109035578526 ,20762 : 37.31077208958969 ,20774 : 34.50670363636866 ,20808 : 39.51807019223759 ,20840 : 34.47357322612912 ,20975 : 47.67271942329194 ,21032 : 35.992335522385574 ,21217 : 33.584771397024504 ,21314 : 29.314876711300226 ,21413 : 28.486891001285077 ,21441 : 29.77132904732899 ,21458 : 33.272793512117374 ,21602 : 30.087139601452026 ,21605 : 33.5917706279455 ,21629 : 38.75246805487901 ,21795 : 34.86657786605935 ,21947 : 27.221764003969607 ,21974 : 38.077336831810335 ,22035 : 35.71311472260257 ,22081 : 33.2801025768748 ,22197 : 33.09619819500052 ,22278 : 42.875404985914564 ,22337 : 25.808366745700237 ,22614 : 32.51112665189707 ,22648 : 32.43178893808731 ,22680 : 33.50304498827974 ,23110 : 30.489149751349863 ,23216 : 40.394143834285416 ,23298 : 40.683443438856315 ,23348 : 34.39396133698931 ,23433 : 37.4728051488558 ,23477 : 29.14806455401764 ,23522 : 24.522648184136866 ,23621 : 25.207947683111037 ,23802 : 32.85967799739707 ,23812 : 29.396929059235443 ,7 : 32.96701445422347 ,16 : 40.51729684779087 ,25 : 24.5746532343384 ,83 : 30.297345114291296 ,172 : 35.10996005183474 ,246 : 29.75306983754645 ,249 : 37.62930593011609 ,251 : 32.18764747232843 ,336 : 35.691557753816234 ,488 : 24.9949932240807 ,507 : 24.212277297908955 ,560 : 33.01779120642202 ,643 : 36.04417690597393 ,707 : 30.651255076472864 ,709 : 31.922624388227284 ,724 : 34.51600835286332 ,731 : 37.258311593136995 ,756 : 35.887013204986616 ,804 : 23.99428621335549 ,1014 : 26.081532458156577 ,1051 : 31.667553226342115 ,1181 : 40.694373718181744 ,1195 : 41.55186501209273 ,1230 : 34.666215093893626 ,1279 : 26.610369390849257 ,1299 : 35.72834339315022 ,1433 : 30.123918364538877 ,1469 : 35.26507773502223 ,1487 : 39.075754479879905 ,1497 : 25.47132635412968 ,1754 : 34.49678908557768 ,1848 : 26.416474946666362 ,1849 : 25.99047189288493 ,1919 : 33.039116475714884 ,1977 : 34.23716283142149 ,2025 : 25.151377124518934 ,2039 : 36.29794114075073 ,2051 : 30.974763169608075 ,2103 : 43.592409822606484 ,2128 : 23.51265594705214 ,2172 : 29.332124739618493 ,2414 : 36.4489624969696 ,2594 : 27.62093635948757 ,2645 : 33.677752238423345 ,2667 : 39.629486598717136 ,2677 : 27.2389472047118 ,2734 : 30.8874309251163 ,2901 : 23.96254439859177 ,3166 : 24.01524367676217 ,3493 : 33.873933794943795 ,3601 : 26.4151559928296 ,3738 : 26.04175820853261 ,3876 : 28.26556312819708 ,3890 : 30.10342421168364 ,3913 : 37.91193725899502 ,3947 : 24.659677574427874 ,3968 : 30.94609412820961 ,4046 : 33.29194407153479 ,4060 : 29.136536821153218 ,4092 : 32.17360907372097 ,4120 : 33.53609129071877 ,4581 : 34.03682656898681 ,4600 : 32.75685860226953 ,4605 : 25.379743835048796 ,4623 : 30.643901306985843 ,4986 : 33.356523488958345 ,5001 : 41.228008342014896 ,5147 : 36.2208210948907 ,5150 : 32.90847478251569 ,5162 : 37.02557219886183 ,5213 : 32.30670044183047 ,5532 : 29.040858370268705 ,5652 : 34.035284614628054 ,5702 : 34.50729516705661 ,5793 : 25.36404784458067 ,6100 : 33.54094596793551 ,6223 : 32.073587649713104 ,6304 : 26.959262259304154 ,6413 : 45.24469018989768 ,6433 : 30.572059505600546 ,6483 : 36.94016820876417 ,6528 : 29.477428363450613 ,6576 : 28.94427728508005 ,6580 : 31.452940883502563 ,6718 : 36.01753376318449 ,6763 : 32.44470107694024 ,6775 : 33.317285719753336 ,6891 : 27.30963184049189 ,6987 : 31.751960633889624 ,7120 : 29.215471862064078 ,7216 : 36.558412721221124 ,7234 : 27.138679606323322 ,7367 : 30.660264125946767 ,7398 : 36.94684178524885 ,7538 : 30.859682258557225 ,7556 : 32.60663361557613 ,7647 : 28.87580354415803 ,7721 : 31.764684600506804 ,7723 : 27.325478991316277 ,7743 : 30.1216841769406 ,7749 : 37.49667000833451 ,7793 : 33.90053471829853 ,7943 : 23.61099093920511 ,7983 : 37.493739389443775 ,8121 : 41.31603571123362 ,8252 : 26.851164127748522 ,8273 : 29.999563542161606 ,8283 : 31.82595259615606 ,8444 : 29.046678942610217 ,8446 : 33.26116350157097 ,8475 : 31.17208934673231 ,8483 : 24.91844827920691 ,8673 : 27.64023448028519 ,8731 : 35.5724341885379 ,8748 : 23.593230589979303 ,8796 : 28.446395647888203 ,8911 : 31.923480519026214 ,8980 : 36.4488430608487 ,9055 : 24.33588819438015 ,9081 : 41.39938473733828 ,9292 : 26.04787470020585 ,9547 : 29.449273274969507 ,9576 : 36.01094735154386 ,9660 : 27.495405383946636 ,9791 : 33.92719682432355 ,9830 : 22.138886759623478 ,9875 : 27.29508529347204 ,9941 : 35.57513137866828 ,9980 : 30.994992270795596 ,10098 : 43.328980129881856 ,10118 : 35.105226375179456 ,10190 : 29.801764092660942 ,10227 : 30.39528136749869 ,10317 : 30.89447665730284 ,10333 : 30.300378356139415 ,10358 : 25.61488662960667 ,10551 : 43.290567629105134 ,10602 : 32.95235342598338 ,10634 : 27.452623418411918 ,10645 : 36.25144215892097 ,10741 : 34.203564897567745 ,10880 : 31.595702401644807 ,10913 : 31.3072513291645 ,11049 : 24.979560451809675 ,11212 : 32.61218509607519 ,11254 : 39.004345508998405 ,11257 : 30.474139106450686 ,11327 : 31.313610582315658 ,11401 : 36.52580440378343 ,11495 : 26.55567601270673 ,11530 : 37.453914648859396 ,11586 : 33.457527716914434 ,11669 : 25.976659323972676 ,11925 : 31.55224808234953 ,12116 : 31.81007981599055 ,12203 : 27.512269265539068 ,12234 : 32.671326891190816 ,12259 : 33.59176395809172 ,12295 : 36.22148189524062 ,12358 : 34.364831472903575 ,12453 : 35.80470472933402 ,12554 : 35.12610777376634 ,12683 : 34.425304503358056 ,12686 : 34.96049523827349 ,12690 : 35.60920322855022 ,12695 : 31.06723546377325 ,12720 : 38.642551688866426 ,12772 : 25.840573838904476 ,12798 : 30.94390449756719 ,12904 : 36.87166029371087 ,12911 : 39.727781288743664 ,12991 : 31.68452155533386 ,13078 : 39.63439020919501 ,13082 : 35.34335163498161 ,13085 : 26.558289098122632 ,13111 : 31.943781618097965 ,13142 : 30.437585559081107 ,13252 : 36.377225758153614 ,13274 : 26.903346360661537 ,13300 : 36.3283851609429 ,13409 : 35.98673069181492 ,13443 : 34.16358213515406 ,13466 : 25.931317020142625 ,13529 : 26.455106321110083 ,13548 : 23.294055735115347 ,13552 : 27.708020602515695 ,13702 : 39.51475769067154 ,13753 : 32.256674509002906 ,13784 : 28.027673066723928 ,13789 : 33.656138027015736 ,13870 : 36.29987809430612 ,14020 : 31.81123048368333 ,14130 : 34.34249435535816 ,14224 : 34.478448604582915 ,14248 : 36.63675720345555 ,14333 : 31.05511561160315 ,14719 : 25.158623677717745 ,14799 : 33.22046370246315 ,14832 : 30.730338779862315 ,14970 : 39.75463152919379 ,15206 : 25.052284688273133 ,15299 : 22.80552778047235 ,15345 : 29.75592676718499 ,15347 : 34.50451054564378 ,15504 : 22.48366025038636 ,15552 : 45.21497346613807 ,15609 : 35.61263111863197 ,15706 : 34.45054543299542 ,15820 : 41.23067056670157 ,15836 : 24.945130943799523 ,15863 : 38.34440665964532 ,16255 : 28.396953509107856 ,16454 : 28.21255223906369 ,16520 : 32.25458802014226 ,16720 : 31.802121541485924 ,16869 : 31.695172947431676 ,16907 : 27.310786196627593 ,16915 : 36.25076249681389 ,17118 : 31.939318349645774 ,17162 : 30.50007128532857 ,17188 : 37.37475921230218 ,17246 : 37.37310426065913 ,17266 : 35.16418029571086 ,17392 : 28.462238398177924 ,17410 : 32.63504586314521 ,17477 : 33.196229984546626 ,17514 : 25.91025284367503 ,17579 : 32.63080452981198 ,17581 : 37.69445248276485 ,17589 : 37.536085885836954 ,17775 : 25.496733225240373 ,17851 : 30.944994865546626 ,17953 : 27.241827437867876 ,18095 : 29.311174903266046 ,18163 : 27.56483552062393 ,18190 : 25.93331301866146 ,18218 : 31.614049030458034 ,18243 : 32.861432199825856 ,18286 : 32.686548962050104 ,18407 : 32.44751419097428 ,18436 : 26.946574910532906 ,18614 : 33.74710951463175 ,18637 : 32.054167483502916 ,18727 : 31.465425051674522 ,18798 : 28.43162447637968 ,18845 : 36.123284107051724 ,18911 : 33.45959869352514 ,19024 : 24.552501174230024 ,19028 : 27.535370323580622 ,19061 : 33.65221853512517 ,19118 : 22.708093490212327 ,19138 : 33.34037007655022 ,19197 : 33.423804343085706 ,19256 : 30.318235758881034 ,19339 : 25.62491507842172 ,19454 : 28.865863009960442 ,19501 : 26.014930733017323 ,19517 : 27.29498232168794 ,19541 : 29.06783715320215 ,19582 : 27.70435299883455 ,19654 : 30.101331508614503 ,19676 : 34.39638783066703 ,19717 : 34.026022079952135 ,19737 : 25.157649670979772 ,19759 : 33.557773482067304 ,19779 : 23.262723531355746 ,19794 : 30.960027539407992 ,19839 : 26.524504534927676 ,19924 : 26.917272493965633 ,19975 : 30.865697354914655 ,20086 : 35.25824170819866 ,20257 : 34.23687377334778 ,20306 : 25.79361776580133 ,20318 : 30.46033516941897 ,20346 : 24.316114477554226 ,20362 : 36.029662249878164 ,20443 : 35.97715799031485 ,20663 : 28.32777922699225 ,20667 : 36.5367924951264 ,20843 : 34.34892562324891 ,20933 : 33.95467586898198 ,20936 : 32.49332023278629 ,20947 : 45.41108127315959 ,20970 : 34.55785327858497 ,20999 : 25.14273022573313 ,21033 : 26.081772014976963 ,21078 : 25.362903418203835 ,21136 : 29.25467790121174 ,21163 : 32.636404342779954 ,21227 : 32.23736571086598 ,21253 : 32.18581409103304 ,21322 : 33.43994868195428 ,21363 : 36.31544635276446 ,21411 : 28.075541002894177 ,21423 : 37.570531324987876 ,21717 : 29.296444988465698 ,21981 : 37.62643249163426 ,21983 : 35.44200871703957 ,22041 : 30.529829051102624 ,22073 : 30.26991293486703 ,22142 : 34.50783024731469 ,22386 : 32.484670013456096 ,22390 : 26.357307470345482 ,22421 : 26.050071889197618 ,22515 : 36.947844483245 ,22711 : 42.94891042663799 ,22851 : 25.628751953519412 ,22878 : 33.6814856107449 ,22960 : 29.785850807724163 ,23028 : 38.08115422356873 ,23116 : 30.29384851338635 ,23165 : 29.8452244264914 ,23186 : 25.273949268905724 ,23275 : 28.78226093705534 ,23309 : 40.579679280545946 ,23446 : 40.86385565974992 ,23514 : 37.25455105942132 ,175 : 25.373126460122304 ,209 : 27.903281858027928 ,236 : 30.252593426492595 ,289 : 30.737035976653726 ,348 : 31.087581452559498 ,502 : 29.274215241163468 ,505 : 32.846631070206115 ,528 : 31.44488165496907 ,639 : 26.398260282558162 ,651 : 31.7761124563416 ,819 : 24.816616290805115 ,1021 : 31.183009112504315 ,1148 : 25.334775085223068 ,1207 : 32.32642890982349 ,1304 : 27.271896582811568 ,1417 : 25.897986596392627 ,1468 : 24.258412588696984 ,1500 : 23.623490262283887 ,1616 : 30.895277650611842 ,1708 : 32.081335566237584 ,1817 : 24.418050175632764 ,1875 : 32.769374623943314 ,1899 : 33.50864051464476 ,1934 : 38.99744485792559 ,2024 : 26.044274723824383 ,2181 : 23.519366805956796 ,2202 : 30.667641292415674 ,2236 : 30.904301362083167 ,2246 : 32.97523048411081 ,2262 : 25.10872894471821 ,2266 : 29.740817329166955 ,2498 : 31.890504371208188 ,2559 : 25.019980279452216 ,2579 : 43.74068814655191 ,2659 : 31.309832631007286 ,2699 : 30.705121486615134 ,2747 : 31.75630771118475 ,2753 : 28.924153723806374 ,2777 : 30.80061672205496 ,2821 : 24.26786901014736 ,2948 : 23.19288582016632 ,3018 : 28.716869582063392 ,3198 : 32.6615152931919 ,3277 : 29.18521026681529 ,3341 : 43.649933118985636 ,3473 : 32.01180162467687 ,3535 : 29.260922271387308 ,3630 : 30.64602339696939 ,3686 : 33.99361169821575 ,3779 : 29.05947358208366 ,3792 : 25.27655213591908 ,3909 : 25.429422334786224 ,3946 : 26.42858555699239 ,3981 : 32.13391906995828 ,4209 : 27.99164776997157 ,4239 : 32.06082412965258 ,4317 : 30.529116888095974 ,4339 : 37.81530269582243 ,4397 : 33.35947340479385 ,4550 : 28.964439084661294 ,4551 : 28.096639543342437 ,4659 : 23.04373371278991 ,4926 : 24.40386102840232 ,4934 : 23.13098881175967 ,4963 : 34.247398218970424 ,5003 : 30.22991167773219 ,5022 : 25.30472537972877 ,5099 : 23.760996787216218 ,5375 : 32.44058049090311 ,5473 : 30.37528133580395 ,5508 : 27.47450196461369 ,5668 : 25.49331069411273 ,5694 : 26.523451916217073 ,5860 : 37.78332008920241 ,5895 : 24.99631247750282 ,5915 : 30.632333775122472 ,5924 : 32.34496271895435 ,5954 : 37.98196685669214 ,6004 : 31.701833441295705 ,6044 : 26.485123093540007 ,6064 : 27.205233450255964 ,6094 : 29.52672895608495 ,6110 : 36.42590383991205 ,6213 : 31.121268300862006 ,6239 : 28.094333115605142 ,6352 : 25.689431618387935 ,6538 : 27.565694655778717 ,6618 : 44.25271065215544 ,6632 : 29.463825440767856 ,6657 : 39.37162072230983 ,6672 : 29.8387026585863 ,6766 : 25.47663295969869 ,6815 : 23.598317546297796 ,6895 : 30.61040000755318 ,7034 : 38.66176843220895 ,7067 : 36.18152711183552 ,7171 : 24.754442529656643 ,7214 : 23.25177377367004 ,7497 : 43.657372010372015 ,7533 : 32.65137064980247 ,7607 : 26.69789181598896 ,7636 : 27.67657101805709 ,7642 : 26.02953878974588 ,7655 : 32.96140099001172 ,7710 : 35.139420415619036 ,7742 : 28.45470227766734 ,7890 : 26.014912874932186 ,7907 : 29.714591387940143 ,7967 : 24.69565416867535 ,7998 : 25.90715664318094 ,8103 : 24.615362781856188 ,8163 : 27.144730391963503 ,8167 : 32.06565867511195 ,8567 : 29.44013565072776 ,8607 : 25.038395335784237 ,8658 : 28.952744966315347 ,8680 : 35.283722322703525 ,8708 : 27.72335421667056 ,8959 : 30.265776292746256 ,9041 : 31.8459015913268 ,9289 : 25.886916418472662 ,9352 : 31.961828231850742 ,9588 : 32.662290910453926 ,9609 : 26.58110401088405 ,9716 : 24.836283297330386 ,9721 : 23.898357493574977 ,9817 : 28.624158882276788 ,9831 : 24.307280531418648 ,9865 : 33.842893461623746 ,10012 : 29.87935872485379 ,10052 : 29.444942488121605 ,10146 : 33.09970422443593 ,10159 : 37.0095353174944 ,10163 : 25.39692475580964 ,10261 : 25.516851052852285 ,10266 : 27.263017370525 ,10287 : 33.2767561340141 ,10399 : 39.71670603923417 ,10495 : 23.257414796390968 ,10569 : 36.701400645124515 ,10660 : 27.460654656652252 ,10743 : 32.0505126597051 ,10750 : 32.99791615446052 ,10853 : 34.5877789816982 ,11004 : 29.648426471951787 ,11013 : 28.550375607466933 ,11026 : 37.65337265958074 ,11101 : 37.09604366402419 ,11193 : 26.726821289913964 ,11261 : 32.47843690857652 ,11311 : 33.69340888364263 ,11548 : 22.81368235786389 ,11595 : 29.800428661281668 ,11651 : 24.230268412619935 ,11882 : 32.49773964125776 ,12040 : 34.02495185773533 ,12045 : 28.276016533240504 ,12178 : 26.609798133506896 ,12220 : 28.531586992683696 ,12221 : 35.76722583980172 ,12272 : 42.23359618502013 ,12285 : 23.87304177763526 ,12436 : 33.91532236701511 ,12440 : 28.506113034562826 ,12471 : 28.621247577579016 ,12502 : 32.96249750910435 ,12517 : 23.972981784711944 ,12574 : 33.06662252617485 ,12578 : 25.949334189925896 ,12687 : 29.260431217221104 ,12811 : 36.34569061294052 ,12834 : 30.171221152765007 ,12939 : 24.273113312710773 ,12963 : 27.972190611622164 ,13087 : 28.954783536358043 ,13105 : 26.28417158651269 ,13114 : 31.47009456622975 ,13149 : 33.32112996368888 ,13193 : 24.327844557921033 ,13530 : 28.34112399614308 ,13572 : 28.699280184663404 ,13663 : 28.99763548362246 ,13770 : 24.121801114786674 ,13822 : 24.243203529303326 ,13831 : 33.048575505676475 ,13836 : 25.75817292298956 ,13890 : 26.639358799860307 ,13970 : 28.483273219126403 ,14042 : 24.608185666121855 ,14043 : 26.817582155502418 ,14227 : 23.027142581579692 ,14253 : 29.62920104043861 ,14292 : 28.18582797897465 ,14306 : 24.093189211513643 ,14338 : 34.59135190966476 ,14422 : 36.37826451131908 ,14441 : 33.39293419241213 ,14497 : 30.838659568009838 ,14538 : 28.297043089132615 ,14642 : 22.151059593115868 ,14948 : 23.136247309897815 ,14949 : 29.643642919464067 ,15180 : 32.72448174864004 ,15314 : 34.02892220456201 ,15449 : 28.86464220553788 ,15456 : 34.118626457422984 ,15471 : 29.5772151878455 ,15474 : 36.61225953664605 ,15549 : 26.16822262561971 ,15599 : 24.631858382266103 ,15668 : 28.465380137550625 ,15694 : 22.96475761845218 ,15777 : 25.77757974502921 ,15780 : 24.28888548711305 ,15786 : 24.96695489817997 ,15803 : 25.989620633624035 ,15925 : 35.82396754707779 ,16023 : 34.36061701236305 ,16103 : 25.657347996904857 ,16246 : 31.27868881950975 ,16292 : 27.028725724982298 ,16483 : 23.27314589411085 ,16533 : 35.06988991317953 ,17048 : 26.977635318240477 ,17223 : 32.131362412746704 ,17273 : 33.715411761866854 ,17284 : 43.77618728336633 ,17385 : 25.489110159346442 ,17444 : 24.484476058492017 ,17519 : 24.263112954052378 ,17547 : 29.872130583536677 ,17613 : 32.06464207017662 ,17631 : 28.266938282975644 ,17664 : 32.657492773643085 ,17761 : 43.986773197850766 ,17791 : 28.29758956425624 ,17804 : 29.343602201089308 ,18130 : 28.840448222026392 ,18159 : 28.96759237572397 ,18173 : 31.481839785455538 ,18240 : 38.64129546293455 ,18312 : 24.673139370519735 ,18323 : 27.496818041421207 ,18382 : 39.84354311478316 ,18417 : 29.379036708152512 ,18443 : 27.028012803165954 ,18502 : 25.55562711823505 ,18553 : 36.83723126869442 ,18758 : 30.100608532558297 ,18761 : 30.57089735504773 ,18833 : 29.319070055447977 ,18881 : 29.48384743079157 ,18946 : 29.8856877193138 ,18954 : 36.55360237040199 ,19047 : 30.320139665849975 ,19077 : 32.793253845671835 ,19184 : 40.35381464437424 ,19232 : 33.93063991004343 ,19296 : 31.442986380452403 ,19345 : 23.337155690725066 ,19628 : 29.089562683096847 ,19767 : 32.93849852668814 ,19830 : 27.06138800004085 ,19870 : 32.564754098467155 ,19906 : 32.68732882278141 ,19907 : 42.981632128889345 ,19978 : 31.13363725845969 ,19983 : 27.55957218386363 ,20087 : 24.344537232293515 ,20133 : 23.510778079464064 ,20252 : 24.434690407611413 ,20312 : 32.57085722605051 ,20388 : 23.539797776054836 ,20412 : 27.378030195400353 ,20414 : 41.13762245156706 ,20431 : 24.775651825275585 ,20467 : 23.321558929428058 ,20508 : 35.482062901506595 ,20570 : 30.707386055935938 ,20604 : 24.01483410405313 ,20616 : 25.026569036354463 ,20652 : 31.751560316913782 ,20655 : 25.731258991734162 ,20714 : 30.82504934848617 ,20961 : 26.04612293235351 ,20971 : 36.44526042133319 ,20988 : 30.639946031066398 ,20990 : 31.556539784277955 ,21127 : 31.60634061255918 ,21216 : 45.299064085249995 ,21228 : 30.427363652907292 ,21252 : 33.736571861670924 ,21367 : 31.32007286555427 ,21484 : 34.19027499411393 ,21586 : 22.283487354947273 ,21635 : 31.786721705937612 ,21698 : 24.728527034712275 ,21792 : 27.67126329800926 ,21839 : 32.21053583153694 ,22008 : 32.24512367171524 ,22013 : 33.901116839361876 ,22122 : 24.701160245402093 ,22172 : 26.983621517706567 ,22501 : 31.75838600661494 ,22575 : 29.680008153511004 ,22595 : 30.184221831389525 ,22693 : 34.47100437638486 ,22728 : 30.520120822379784 ,22760 : 31.015250713221384 ,23112 : 31.229360764903234 ,23137 : 24.813554435808197 ,23147 : 31.176068840652132 ,23200 : 30.847099811299984 ,23214 : 29.565152268010518 ,23233 : 35.14601866188083 ,23267 : 25.76292425262192 ,23483 : 32.0168751692503 ,23551 : 41.15861725295799 ,23680 : 32.86559803176584 ,23700 : 25.663935542741466 ,63 : 27.34636900445458 ,161 : 33.359108729578985 ,218 : 38.143057036898824 ,428 : 54.30042030725484 ,430 : 46.283169793820356 ,513 : 27.362103341968872 ,567 : 29.24079255532785 ,602 : 32.448706327864784 ,622 : 40.573788024894355 ,670 : 38.74296814637898 ,692 : 39.6875768829335 ,750 : 38.39684320480033 ,874 : 28.18910723780635 ,906 : 29.37859727964692 ,940 : 40.201969228108254 ,1096 : 30.993310933645137 ,1171 : 28.090803753897944 ,1229 : 25.323643776845664 ,1411 : 43.261186284598715 ,1434 : 31.477006341028304 ,1568 : 34.11439286085933 ,1610 : 36.74350175097993 ,1647 : 28.03660364390744 ,1724 : 28.060378304132747 ,1757 : 42.90680880235469 ,1774 : 37.54265065377478 ,1914 : 37.46973214998044 ,2141 : 31.899325240100296 ,2301 : 27.59601336347732 ,2402 : 46.224433982838846 ,2515 : 36.95232147673492 ,2738 : 34.303384422531884 ,2752 : 43.451038879663756 ,2800 : 32.46696074180914 ,2905 : 37.4852581350451 ,2919 : 27.60236779270151 ,2943 : 38.859231667340076 ,2965 : 26.222641014315094 ,3175 : 36.15714197746014 ,3186 : 30.305322816750635 ,3194 : 32.61952343492573 ,3244 : 26.328352120424054 ,3249 : 38.19002746923793 ,3260 : 35.19903114123785 ,3301 : 41.13642955616668 ,3313 : 35.25695700995882 ,3467 : 38.35350895886194 ,3655 : 48.18132390017234 ,3662 : 42.067371898384366 ,3749 : 28.57438878515531 ,4039 : 44.1175953897631 ,4152 : 40.472912442414014 ,4194 : 37.18876920325308 ,4319 : 29.325697202000924 ,4326 : 27.446670889202768 ,4348 : 26.760802123058756 ,4405 : 35.97927435052255 ,4432 : 30.303268839617175 ,4468 : 33.89168162824707 ,4538 : 27.481143581655314 ,4552 : 35.955130065239246 ,4621 : 31.659590808074555 ,4706 : 40.35392743695123 ,4861 : 34.54074401928008 ,4947 : 39.79745090254538 ,4992 : 33.872180455467394 ,4995 : 32.86169179526763 ,5018 : 27.708788843982028 ,5038 : 27.480229257343254 ,5244 : 26.79971749800041 ,5309 : 27.67654096847624 ,5451 : 26.837488177108593 ,5461 : 31.846803052854444 ,5773 : 67.64078928969805 ,5840 : 36.47808498844339 ,5864 : 44.80066951749763 ,5939 : 27.331061687112406 ,6056 : 41.28594967509805 ,6059 : 36.629984426949136 ,6079 : 33.90827983050353 ,6168 : 40.54060331379161 ,6230 : 33.9915790880371 ,6325 : 37.23898812334327 ,6337 : 30.934045683810513 ,6349 : 37.0768296435619 ,6477 : 37.42157200904856 ,6490 : 39.38298273216196 ,6537 : 33.50666920210418 ,6557 : 37.716897478038646 ,6565 : 48.25091486421444 ,6753 : 35.34004586238734 ,6930 : 36.489467656287864 ,7019 : 27.712712628003786 ,7501 : 37.12885982140467 ,7592 : 36.788493386380544 ,7612 : 30.041710809431308 ,7680 : 37.69384214831359 ,7753 : 27.72737504414371 ,7799 : 37.8765005266468 ,7840 : 45.93546747539239 ,7942 : 40.442667508815916 ,7984 : 37.851288143404624 ,8062 : 35.678508403667045 ,8184 : 30.445006843187432 ,8211 : 30.128701063951876 ,8266 : 41.700520228257425 ,8291 : 31.572091584638382 ,8404 : 40.64653879908911 ,8495 : 36.37310689615503 ,8695 : 40.45730554663966 ,8803 : 32.66600350468461 ,8942 : 31.259848753710934 ,9013 : 27.981304266359253 ,9025 : 32.82475479500304 ,9053 : 43.28508245279134 ,9229 : 41.60359119171832 ,9371 : 38.103630336560855 ,10085 : 35.769837370657626 ,10124 : 46.10354120820962 ,10165 : 45.05126260304195 ,10185 : 42.314059233424494 ,10242 : 38.29782326024307 ,10248 : 37.20314878808324 ,10283 : 35.17431686206144 ,10361 : 30.549011186515735 ,10435 : 36.73291691987004 ,10546 : 26.323934174715276 ,10670 : 43.31275772935887 ,10776 : 33.70594791447716 ,10812 : 40.260627519820915 ,10863 : 36.96710759886701 ,10969 : 37.78975246762255 ,11141 : 39.26574915591271 ,11213 : 33.96382525176866 ,11258 : 39.57034069023171 ,11304 : 41.42640418372708 ,11313 : 28.255045828492708 ,11337 : 29.325936292219062 ,11414 : 39.62000479201278 ,11890 : 34.99278693765337 ,11929 : 37.18448710821579 ,12127 : 27.69456611512465 ,12146 : 27.709023413692954 ,12418 : 29.374761024814426 ,12432 : 39.34273731540918 ,12501 : 40.37989207791741 ,12520 : 38.045420208515225 ,12617 : 32.08770389892159 ,12648 : 37.21624752541162 ,12733 : 34.0694192610428 ,12787 : 43.427279411702386 ,12923 : 39.9547681360438 ,12955 : 37.343130159012 ,13011 : 35.95506521840658 ,13229 : 41.798380983409636 ,13309 : 29.649166367239136 ,13347 : 36.76761224864757 ,13352 : 28.14686913785525 ,13419 : 30.629559115841587 ,13518 : 37.12786863414432 ,13722 : 46.911912088668444 ,13729 : 26.636967093907646 ,13932 : 29.48488998873492 ,13959 : 37.1578902639932 ,13997 : 34.3565871480369 ,14029 : 27.537293608819592 ,14102 : 40.887256647785364 ,14106 : 31.621343131275093 ,14265 : 34.51687821518873 ,14351 : 30.653511787907163 ,14383 : 29.88734583478277 ,14393 : 42.814373220435556 ,14448 : 26.35483525383884 ,14461 : 40.686019266933044 ,14485 : 39.789694160608924 ,14523 : 37.80756182814664 ,14648 : 25.70208402117776 ,14728 : 32.94675614325348 ,14745 : 27.824603125014306 ,14804 : 37.489705760981224 ,15042 : 38.32377182886767 ,15209 : 53.456950696244895 ,15272 : 30.417778447173628 ,15323 : 31.083764234038615 ,15436 : 41.95360366948551 ,15562 : 44.86901890684186 ,15626 : 29.247023384116268 ,15645 : 36.259866226934925 ,15877 : 28.795024971563233 ,16166 : 26.311321348478508 ,16198 : 31.39365223951632 ,16235 : 38.71709520515638 ,16239 : 45.044688799930825 ,16260 : 36.91768936672225 ,16383 : 30.606469373902836 ,16457 : 31.21074678359713 ,16605 : 46.32469876186129 ,16713 : 50.909110647088355 ,16717 : 37.414585159475116 ,16801 : 33.153700124890506 ,16856 : 41.47575209232876 ,16996 : 31.904266722746417 ,17171 : 36.64419230892718 ,17177 : 36.696711645223274 ,17196 : 25.78682478592531 ,17215 : 37.65492734688078 ,17240 : 30.161500208142936 ,17333 : 36.46587106693406 ,17353 : 32.59222156491294 ,17382 : 27.56668571034717 ,17419 : 31.443097239712625 ,17573 : 39.78897847611746 ,17608 : 30.437448556127624 ,17611 : 32.653158127025634 ,17617 : 37.650978335747816 ,17692 : 41.59672296393464 ,17816 : 44.32402742881376 ,17992 : 51.413003231892475 ,18027 : 37.612177666446 ,18074 : 33.244583704432046 ,18120 : 26.963098478657876 ,18378 : 43.04116892531893 ,18581 : 39.58028634085381 ,18605 : 28.625372020363386 ,18834 : 30.05803685608055 ,18847 : 33.15719887003727 ,18853 : 48.84931938330737 ,18870 : 38.16725126680651 ,18877 : 28.441575254844434 ,18916 : 34.01595276748838 ,19048 : 32.576747655881775 ,19381 : 29.378286327302266 ,19400 : 42.70858458482082 ,19407 : 37.60058883172626 ,19440 : 38.870394084277244 ,19562 : 28.040190366901335 ,19631 : 38.367877975660704 ,19763 : 33.26653185716871 ,19901 : 40.98262745719976 ,19936 : 37.39247608740276 ,19950 : 32.907441501216496 ,20111 : 28.327889798746437 ,20351 : 28.143248617827695 ,20363 : 27.92683394787594 ,20379 : 27.758033195458825 ,20418 : 35.295091644085765 ,20425 : 27.42180958222801 ,20439 : 31.199670340685255 ,20456 : 46.20910662988007 ,20534 : 42.11753411638592 ,20693 : 27.63252764690472 ,20753 : 35.90930359337343 ,20860 : 30.879477594497164 ,21111 : 28.20322613158864 ,21133 : 26.435808517211196 ,21174 : 38.06197521697133 ,21176 : 42.289220924836535 ,21427 : 28.686095482165644 ,21524 : 27.977967972361135 ,21561 : 31.428300903990337 ,21576 : 28.80388585866961 ,21758 : 31.536135513764542 ,21778 : 32.01357511630513 ,21779 : 27.329351805021933 ,21910 : 38.0561372143215 ,22129 : 32.81768680045056 ,22185 : 33.67769346269317 ,22391 : 39.54855086295463 ,22567 : 27.682306326066005 ,22619 : 27.45678946814958 ,22641 : 47.73174262542967 ,22754 : 43.782990094505294 ,22831 : 34.165381490110995 ,22888 : 37.073131809135475 ,22948 : 36.2277527235243 ,23031 : 38.92538679405326 ,23154 : 33.71589010389752 ,23197 : 33.80645192448777 ,23204 : 32.54802051471218 ,23238 : 38.80543712568835 ,23350 : 35.530924777027785 ,23400 : 31.26757093284392 ,23558 : 35.902308847615025 ,23727 : 36.08047668995816 ,23750 : 30.92029927506042 ,23776 : 42.3165378023578 ,44 : 27.988691876563944 ,47 : 26.493303130577086 ,76 : 34.401193222406285 ,86 : 33.39149860509587 ,255 : 36.74353132245687 ,414 : 32.033842742862504 ,434 : 27.016408361679378 ,525 : 43.49622399800981 ,581 : 38.5369744533861 ,690 : 32.82895232184789 ,728 : 44.403114493635 ,928 : 30.990911775035112 ,945 : 43.17139413987743 ,1067 : 38.48616705062688 ,1233 : 25.758596573006926 ,1251 : 32.27874150827884 ,1265 : 32.38579885169983 ,1516 : 39.27290286099849 ,1541 : 28.136290446653845 ,1578 : 32.21612608300521 ,1746 : 25.25757396589032 ,1856 : 42.492651534276455 ,1915 : 36.38389186417491 ,2033 : 45.54428882337997 ,2132 : 25.26974365740441 ,2175 : 39.35788522144911 ,2200 : 36.57629627350737 ,2258 : 25.83594187788192 ,2313 : 40.39508956734499 ,2324 : 34.84874455018639 ,2478 : 33.03140184936301 ,2508 : 32.05858050519382 ,2597 : 31.99959759658908 ,2794 : 35.4107061343749 ,2869 : 35.6029885288209 ,3060 : 30.040164421822983 ,3250 : 38.36867764203858 ,3302 : 26.369865698819684 ,3325 : 26.385811346023875 ,3428 : 28.894319083107202 ,3570 : 33.06535375612758 ,3614 : 35.85883222826281 ,3667 : 41.44199355467041 ,3695 : 36.39047316442182 ,3744 : 35.69006180749908 ,3764 : 38.77392799420429 ,3775 : 38.977631014405276 ,3787 : 44.162058886538176 ,3914 : 26.094168785873247 ,3955 : 27.6066605205663 ,4068 : 39.02298774311855 ,4160 : 28.182759112435058 ,4191 : 36.969656015756755 ,4216 : 48.011620742303094 ,4351 : 31.650916531714188 ,4366 : 40.48568295904418 ,4518 : 38.76388386798404 ,4564 : 27.24472265598368 ,4754 : 41.01076607956567 ,4872 : 32.66969976377768 ,4951 : 33.900209038853134 ,4961 : 42.55428765348027 ,4966 : 33.17936154777858 ,5006 : 40.83584629222151 ,5100 : 35.24469080742449 ,5142 : 40.804146258511494 ,5157 : 27.65595126849988 ,5163 : 38.89209359070264 ,5205 : 46.09679826924029 ,5216 : 33.187599126065976 ,5280 : 40.231619420822334 ,5485 : 26.808192955528238 ,5720 : 38.61228466880551 ,5844 : 41.320909907951986 ,6040 : 42.72604510939224 ,6047 : 27.15031791580421 ,6084 : 27.42977339554483 ,6247 : 40.71910948963482 ,6261 : 31.938951961417978 ,6434 : 30.012344171377997 ,6441 : 29.61711125234457 ,6829 : 28.257405906347657 ,6849 : 34.83027273867088 ,6928 : 40.49573015591661 ,7031 : 42.18072657411941 ,7061 : 29.004753185920315 ,7071 : 31.249542752474753 ,7127 : 39.2655068214508 ,7197 : 26.207529859016134 ,7266 : 40.116397629161106 ,7371 : 40.53671979992484 ,7619 : 30.820766021964648 ,7740 : 39.523125586229995 ,8020 : 25.883262339810297 ,8038 : 34.18858351833715 ,8125 : 35.06822948450102 ,8270 : 27.88037645071367 ,8334 : 37.644868657673655 ,8465 : 28.205817134114906 ,8539 : 34.106562206364515 ,8753 : 39.91653247064939 ,8933 : 39.32141757646809 ,9043 : 35.83933140085993 ,9150 : 36.907392861666125 ,9153 : 46.810071760874415 ,9161 : 29.595164215140837 ,9192 : 26.545393067940502 ,9220 : 32.22595609390329 ,9223 : 33.356251650854944 ,9247 : 39.70139195781787 ,9255 : 36.74763842553234 ,9304 : 41.670458247785895 ,9453 : 41.057954371898994 ,9493 : 29.430615146723756 ,9503 : 38.02172516898168 ,9718 : 41.99127913435589 ,9724 : 28.268271603558894 ,9821 : 44.55317064917158 ,9917 : 35.55879132024908 ,9923 : 30.495411843895198 ,9939 : 31.001957258644374 ,10069 : 45.837434572693454 ,10125 : 31.148218635885257 ,10126 : 42.55004183749659 ,10211 : 38.90679146540602 ,10282 : 43.811364555703754 ,10295 : 42.907191596670856 ,10346 : 31.88843526376437 ,10410 : 35.033469249799815 ,10611 : 35.68115005479823 ,10631 : 37.14094754300929 ,10641 : 38.353379664207615 ,10809 : 33.220133632871104 ,10973 : 25.75583278402074 ,11072 : 32.406825569237945 ,11091 : 35.420024807318626 ,11306 : 29.134595111698967 ,11365 : 35.829949340303145 ,11392 : 35.00893974572256 ,11415 : 51.530818831726755 ,11476 : 35.38151500837269 ,11539 : 32.68381093077191 ,11607 : 35.37428903748462 ,11611 : 28.892056278795533 ,11652 : 43.94227577645395 ,11686 : 30.362676988764775 ,11712 : 43.36137005658586 ,11789 : 40.933397841729125 ,12008 : 33.041958175110985 ,12073 : 37.287780602600115 ,12134 : 36.796328778210494 ,12142 : 25.771124412814142 ,12333 : 26.23295209235683 ,12366 : 34.708155055813265 ,12488 : 34.16411711828854 ,12490 : 28.564630474275372 ,12749 : 25.845111179840984 ,12780 : 34.102422845263845 ,12781 : 39.185095415643296 ,12824 : 36.18101187541785 ,12886 : 34.14424369013711 ,13174 : 27.912450997191247 ,13178 : 37.06210451928017 ,13303 : 45.12070794546823 ,13740 : 31.534478229430963 ,13775 : 47.547275129625085 ,13938 : 31.86381641058489 ,13967 : 34.73676813661475 ,13987 : 29.830909027058855 ,14369 : 35.007172780146895 ,14377 : 40.381161384913604 ,14459 : 27.785820037265363 ,14491 : 38.177362780861316 ,14521 : 36.36856192890022 ,14547 : 34.43261980236818 ,14615 : 31.413102119752754 ,14683 : 28.46238689185747 ,15004 : 42.001109965462994 ,15244 : 34.66409336974079 ,15422 : 27.812004546049625 ,15435 : 26.94523901494684 ,15458 : 38.17935122266014 ,15752 : 31.445431890118975 ,15771 : 37.31352395583686 ,15930 : 32.879652746014614 ,15991 : 49.861686823100236 ,16024 : 46.775362895580265 ,16076 : 33.79268163340359 ,16086 : 36.56817750795434 ,16116 : 45.56270895236204 ,16187 : 36.67131898700236 ,16249 : 34.90246767324013 ,16357 : 26.610052357516302 ,16378 : 29.808873106125283 ,16530 : 30.256080341329334 ,16544 : 44.40623809635734 ,16576 : 41.28515169676185 ,16634 : 40.23217225674591 ,16721 : 36.11391767822732 ,16789 : 30.788859507061904 ,16844 : 41.89566254832529 ,16867 : 35.1079191281308 ,16947 : 32.26036717280074 ,16970 : 42.69321440343419 ,17039 : 35.843338167990794 ,17374 : 37.78077129684295 ,17383 : 31.534478229430963 ,17545 : 44.46593621009797 ,17684 : 33.9028562864138 ,17776 : 38.617160793688306 ,17862 : 31.05016374145692 ,17866 : 38.5291192515078 ,17993 : 28.143975265051367 ,18049 : 46.63673821551423 ,18063 : 30.174676405852395 ,18075 : 40.05834305753655 ,18144 : 46.74032086143989 ,18226 : 29.789791033092524 ,18424 : 33.824807826800416 ,18428 : 25.151844969270844 ,18509 : 31.545358085424688 ,18511 : 29.053052510805298 ,18574 : 39.24609042364326 ,18617 : 40.490770656655805 ,18699 : 35.491820867976486 ,18732 : 37.77499277388808 ,18910 : 38.01532620387064 ,18975 : 43.460024787990356 ,19093 : 46.12775629412465 ,19177 : 30.16143057774338 ,19185 : 40.621141837869224 ,19201 : 34.36448214718565 ,19283 : 29.459111530445494 ,19402 : 36.70384061658551 ,19532 : 28.799561781287593 ,19556 : 33.97393935483583 ,19567 : 27.48117796001535 ,19692 : 36.500769885898464 ,19921 : 36.90030501312489 ,19947 : 53.90505654405633 ,20067 : 27.04146286032945 ,20084 : 44.54343271433372 ,20121 : 31.072474417524255 ,20433 : 26.345274317840932 ,20905 : 41.503316808632114 ,21040 : 47.81793693223546 ,21069 : 26.25031219934928 ,21101 : 41.5705258838307 ,21296 : 27.64103156977346 ,21321 : 43.88374383147792 ,21352 : 35.67401849680406 ,21475 : 25.00447078348625 ,21498 : 28.081689886095532 ,21501 : 35.65149577677815 ,21608 : 38.793332566464116 ,21676 : 28.468775116588738 ,21822 : 45.41008096043977 ,21846 : 25.4822118985468 ,22179 : 43.007175582586164 ,22211 : 25.143654504345676 ,22279 : 47.78588050034993 ,22338 : 27.407256399961206 ,22350 : 28.52245756322961 ,22445 : 36.2784295589148 ,22463 : 31.32002311112526 ,22726 : 34.572993939885336 ,22860 : 31.096771600765667 ,23076 : 25.503531720619204 ,23128 : 32.879911755715455 ,23144 : 36.95350538859908 ,23207 : 38.00325600701419 ,23245 : 32.22045842701415 ,23361 : 41.326685513884605 ,23420 : 38.879685440849094 ,23456 : 36.335442370564294 ,23595 : 52.458670830714105 ,23778 : 35.589300835657596 ,199 : 35.916995040056484 ,397 : 34.33141582833643 ,420 : 30.163610428806358 ,469 : 29.084364479988707 ,591 : 34.31200220731391 ,594 : 35.489432250372175 ,673 : 29.6317289184924 ,689 : 36.957612360028094 ,883 : 33.500788607358025 ,1015 : 40.59871472141067 ,1121 : 32.661648295148524 ,1297 : 31.227078095639044 ,1475 : 45.974709175881046 ,1650 : 51.31201144985667 ,1755 : 28.470956620361044 ,1953 : 34.04144388121234 ,2008 : 26.604476451111314 ,2019 : 35.700858954398214 ,2131 : 28.599350350821105 ,2211 : 30.52116370665568 ,2318 : 35.673825354704235 ,2337 : 26.728217379042583 ,2385 : 27.04349895720799 ,2409 : 41.68992478322376 ,2428 : 51.13193170143729 ,2599 : 27.991962944114363 ,2861 : 27.158793492628835 ,2988 : 34.3778732523321 ,3245 : 31.47717182165364 ,3258 : 31.860338147204352 ,3281 : 30.57788693994161 ,3403 : 28.781000568745966 ,3583 : 47.35482652186653 ,3824 : 37.03849539680129 ,3970 : 27.566355029139537 ,3983 : 27.495763656485664 ,4014 : 35.36502051677015 ,4051 : 34.637569782340904 ,4052 : 36.34671262432739 ,4065 : 41.30531230776572 ,4146 : 33.09645206772474 ,4214 : 31.620383530267524 ,4222 : 37.4947659133103 ,4291 : 50.127906785099135 ,4336 : 42.34972932954584 ,4340 : 33.65486780969695 ,4375 : 33.39293454747882 ,4456 : 28.846593405188507 ,4515 : 27.42136733698871 ,4555 : 34.25107434817645 ,4708 : 33.14124295562243 ,4839 : 31.61338392172708 ,5174 : 30.117620656791384 ,5204 : 28.949776793534927 ,5206 : 31.080544704374812 ,5504 : 33.841643831755405 ,5607 : 30.13591620722587 ,5650 : 30.991222442134294 ,5655 : 33.22474279119539 ,5690 : 32.70000319568586 ,5712 : 26.823615621564525 ,5795 : 32.300237205850024 ,5828 : 26.617438233650006 ,5856 : 32.979367340973944 ,5863 : 29.978255549484157 ,5937 : 26.716034653755806 ,6075 : 27.06653668071839 ,6150 : 37.053748659403055 ,6344 : 45.77023485817141 ,6598 : 34.8335326154544 ,6682 : 29.95553330062408 ,6719 : 34.346588527569246 ,6772 : 32.44414951228365 ,6777 : 35.23098148810274 ,6950 : 29.10121779132316 ,7036 : 33.4587958135085 ,7163 : 42.526850713780846 ,7230 : 46.06123575816448 ,7417 : 28.03172204439049 ,7541 : 36.43572025338097 ,7594 : 33.16707213964262 ,7611 : 30.30227765985266 ,7713 : 27.549239805932334 ,8123 : 36.17521443059423 ,8208 : 29.09715265819263 ,8238 : 33.79477451486525 ,8356 : 37.66392795972911 ,8425 : 35.26102821669531 ,8434 : 28.117474074758384 ,8471 : 29.470178800497077 ,8606 : 32.63641121524573 ,8723 : 36.14206242791042 ,8726 : 34.42978137700807 ,8800 : 28.15963286072792 ,8829 : 31.78507193370872 ,8901 : 37.503913374885045 ,9012 : 28.588954117498098 ,9094 : 32.57018304184355 ,9230 : 45.0189332077405 ,9254 : 29.15327934585025 ,9301 : 25.712628210953078 ,9555 : 31.80999949607271 ,9570 : 37.727375370462596 ,9603 : 35.29842590091731 ,9622 : 33.095957587366115 ,9888 : 36.089918717471676 ,9895 : 36.45771144863815 ,9912 : 33.51496048920991 ,9918 : 46.850538653011405 ,9976 : 30.065771113275353 ,10049 : 33.76463465927898 ,10116 : 31.853091962789765 ,10138 : 28.561433991983975 ,10309 : 30.84973035708804 ,10328 : 34.06729530651283 ,10392 : 49.20671760356868 ,10453 : 28.07556179071738 ,10530 : 34.75048650466605 ,10594 : 35.37119492241031 ,10728 : 27.44503998027596 ,10738 : 54.77303590910004 ,10754 : 39.74477155654297 ,10765 : 38.96197876919997 ,10888 : 35.046666146117325 ,10987 : 27.445369183961937 ,11031 : 37.98987678283737 ,11062 : 35.83735753478853 ,11092 : 32.745668679504945 ,11110 : 36.1764326507856 ,11151 : 34.21551257721342 ,11201 : 40.661482516971674 ,11256 : 44.158739169130556 ,11483 : 30.7552790006941 ,11684 : 25.54574548280387 ,11840 : 28.222461001291045 ,11860 : 46.56768288682984 ,11906 : 30.711435607728966 ,11967 : 35.79372640084853 ,12114 : 36.20601203231029 ,12307 : 26.888463864157657 ,12324 : 32.87744037113828 ,12348 : 27.34407357056883 ,12376 : 42.968402454965414 ,12605 : 33.06825054034618 ,12779 : 33.249371980084256 ,12803 : 31.053069965971925 ,12829 : 35.64572009716252 ,12883 : 30.93023873326045 ,12885 : 32.25020346171586 ,12914 : 30.032853756078143 ,12915 : 26.48270122848108 ,13077 : 48.69379428212801 ,13153 : 30.97407675314663 ,13165 : 32.28382382946815 ,13177 : 35.22452585758016 ,13187 : 39.36665877997719 ,13197 : 27.039003366189103 ,13241 : 34.007052836112976 ,13257 : 26.081716150420853 ,13383 : 49.300036702134115 ,13433 : 28.345335498684992 ,13491 : 26.684486889958432 ,13703 : 32.03089302755233 ,13710 : 35.80831142183398 ,13888 : 31.554637879342486 ,13942 : 40.00993740966586 ,13988 : 35.721089079197576 ,14429 : 34.803916692591315 ,14484 : 36.40942929101227 ,14507 : 33.442223854871386 ,14571 : 34.28927097311177 ,14698 : 37.644172437834214 ,14838 : 47.5419171009199 ,14961 : 30.582330416244687 ,14981 : 29.676178152192364 ,15032 : 27.050579117106878 ,15047 : 28.282573069854294 ,15087 : 26.371476427210027 ,15200 : 36.443546598869425 ,15230 : 26.52953153674702 ,15259 : 25.76442233394233 ,15261 : 29.97753107501543 ,15431 : 35.656344223822586 ,15466 : 34.402254935730774 ,15749 : 28.521337216877427 ,15768 : 32.54068894417607 ,15810 : 38.38430345975103 ,15875 : 28.422871283031373 ,15959 : 47.36194571155068 ,16014 : 32.60196223965738 ,16226 : 27.141610476088747 ,16242 : 34.60551920007824 ,16360 : 33.25516956983837 ,16578 : 33.89462421595816 ,16603 : 33.35327204388897 ,16762 : 34.457270805544006 ,16825 : 31.173689481662606 ,16883 : 47.4720238430327 ,16955 : 31.755869652360246 ,16964 : 40.95518581013495 ,17198 : 36.00623519830958 ,17242 : 41.48242793672085 ,17568 : 53.962409646992 ,17591 : 32.92940752121763 ,17715 : 34.68668717715227 ,17762 : 32.2760929136181 ,17859 : 34.939477232454855 ,17917 : 35.13881806016206 ,17943 : 34.14268021938341 ,18098 : 31.90134644864641 ,18115 : 27.24275711831002 ,18131 : 41.398163734213135 ,18174 : 32.63018419124484 ,18198 : 34.620360352587355 ,18204 : 25.931463082129977 ,18351 : 35.00641624393129 ,18395 : 29.53428366622675 ,18409 : 34.86434958013751 ,18588 : 38.87943580308643 ,18590 : 29.648973496581 ,18660 : 33.64845198830932 ,18755 : 36.78249658921555 ,18830 : 32.690532802116664 ,18876 : 27.931752872708188 ,18884 : 28.506160934328534 ,18932 : 28.00357292565589 ,19151 : 54.95082863670164 ,19321 : 47.948353419418076 ,19329 : 35.30828419044034 ,19405 : 45.47521787115662 ,19483 : 35.842059788798124 ,19549 : 34.128877851275824 ,19640 : 29.58646833559467 ,19732 : 49.59198544362235 ,19847 : 41.50564651901478 ,19868 : 36.01418554448026 ,20237 : 38.793263819339785 ,20355 : 28.25123881780308 ,20374 : 25.94465409678593 ,20553 : 36.304293987386075 ,20608 : 36.90202243957809 ,20622 : 37.414512512652415 ,20632 : 33.01718875895045 ,20669 : 35.87276432208677 ,20785 : 35.91536532826462 ,20883 : 40.29541275365366 ,20903 : 36.59565903818033 ,20917 : 27.600813430036393 ,20923 : 29.570623764562566 ,20966 : 29.508637542402848 ,20989 : 34.5366164984605 ,21265 : 37.80609151229451 ,21331 : 35.254012862202494 ,21520 : 28.903159703004214 ,21583 : 27.595651902780297 ,21680 : 37.571906952238834 ,21753 : 44.10565230333243 ,21848 : 34.574920842245575 ,21853 : 43.33819300644657 ,22191 : 40.10119308736139 ,22342 : 28.55233264628287 ,22362 : 33.97159239813349 ,22380 : 33.163143174446546 ,22423 : 26.345892159402478 ,22487 : 27.611271280257395 ,22539 : 32.2548126737935 ,22572 : 31.87972289234864 ,22665 : 35.45332690645571 ,22765 : 34.6098128294751 ,22901 : 37.379885255604236 ,23003 : 42.036972930344454 ,23060 : 28.084901635548697 ,23075 : 26.191916933096174 ,23097 : 34.020496217806766 ,23121 : 26.519113409167417 ,23180 : 34.09999079038654 ,23270 : 35.39606679629 ,23294 : 37.16833969565608 ,23434 : 36.20688899986877 ,23550 : 38.00596304130425 ,23648 : 39.868867147517754 ,23733 : 39.52472430903841 ,23755 : 34.38307883918471 ,23782 : 35.49304952977015 ,23791 : 33.88790472134676}

for k, v in ind_d_di.items():
    if k in D1_24:
        D1_24_li.append((k, v))
    if k in D1_48:
        D1_48_li.append((k, v))
    if k in D1_72:
        D1_72_li.append((k, v))
    if k in D2_24:
        D1_24_li.append((k, v))
    if k in D2_48:
        D1_48_li.append((k, v))
    if k in D2_72:
        D1_72_li.append((k, v))
        

train_preds3 = train_preds2.copy()
cnt = 0
for li, index_li in [[D1_24_li, D1_24], [D1_48_li, D1_48], [D1_72_li, D1_72], [D2_24_li, D2_24], [D2_48_li, D2_48], [D2_72_li, D2_72]]:
    for tmp in li:
        i = tmp[0]
        d = tmp[1]
        vec1 = temp_X[i]
        for j in index_li:
            if i == j:
                continue
            vec2 = temp_X[j]
            d = distance.euclidean(vec1, vec2)
            if d <= max_d*0.8:
                print(i, j)
                train_preds3[j] = tag_y
                cnt+=1
                if cnt >= 10:
                    break
    

In [ ]:
t = train_preds.copy()
print(metric(y[:,:tag_size], t[:,:tag_size]))
t_ = train_df[train_df["cp_type"] == 0]
t[t_.index] = np.zeros((t_.shape[0], t.shape[1]))
t = np.where(t > 1, 1, t)
print(metric(y[:,:tag_size], t[:,:tag_size]))

In [ ]:
li = []
not_li = []
for i in range(y.shape[0]):
    for j in range(y.shape[1]):
        if y[i][j] == 1:
            #print("====={}, {}====".format(i,j))
            rank = np.where(train_preds[i].argsort()[::-1] == j)[0][0]+1
            #print("rank {}".format(rank))
            if rank <= 20:
                li.append(j)
            else:
                not_li.append(j)
            #print(train_preds[i][j])

In [ ]:
li = list(set(li))
not_li = list(set(not_li))
for i in li:
    if i not in not_li:
        print(i)

In [ ]:
cols = pd.read_csv("../Data/Raw/train_targets_nonscored.csv").columns
pd.read_csv("../Data/Raw/train_targets_nonscored.csv")[cols[256-206]]

In [ ]:
        # GP
        #print("gp")
        #function_set = ['add', 'sub', 'mul', 'div',
        #                'sqrt', 'log', 'abs', 'neg', 'inv',
        #                'max', 'min']
        #gp_ncomp = 20
        #gp = SymbolicTransformer(
        #    generations=40, 
        #    population_size=4000,
        #    hall_of_fame=100, 
        #    n_components=gp_ncomp,
        #    function_set=function_set,
        #    parsimony_coefficient=0.0005,
        #    max_samples=0.9, 
        #    verbose=0,
        #    random_state=seed, 
        #    n_jobs=2)


        #gp_cols = [f"gp-{i}" for i in range(gp_ncomp)]
        #gp = gp.fit(train_X[GENES_], train_y[:,71])
        #train_X[gp_cols] = gp.transform(train_X[GENES_])
        #valid_X[gp_cols] = gp.transform(valid_X[GENES_])
        #test_X[gp_cols] = gp.transform(test_X[GENES_])
        #pub_test_X[gp_cols] = gp.transform(pub_test_X[GENES_])